In [1]:
# Import needed dependencies
import requests
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from datetime import date
from bs4 import BeautifulSoup, Comment

In [2]:
today = date.today()

# dd/mm/YY
current_year = today.strftime("%Y")

last_year = int(current_year) - 1

In [3]:
last_five_years = []
for i in range(0,5):
    last_five_years.append(int(current_year) - i)

In [4]:
# Create a list to help create a dataframe from batter statistics data
nba_stats = []

# Create a loop to create a dataframe from Basketball Reference tables
for year in last_five_years:
    
    # input URL and use BeautifulSoup to parse through the page
    url = f'https://www.basketball-reference.com/leagues/NBA_{year}_totals.html'
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')

    # Grab the table element that has NBA player statistics
    table = soup.select_one('#div_totals_stats') 

    # Grab data from table and put it into the list created above
    for tr in table.select('tr:has(td)'):
        tds = [td.get_text(strip=True) for td in tr.select('td')]
        tds.append(year)
        nba_stats.append(tds)

In [5]:
# Create dataframe for batter statistics
nba_stats_df = pd.DataFrame(nba_stats)

# Create an empty list to store dataframe header information
header_list = []

# Grab the table header information to use as column headers in our dataframe
for tr in table.select('tr:has(th)'):
    ths = [th.get_text(strip=True) for th in tr.select('th')]
    header_list.append(ths)

# For loop returns a list of lists, and we only need the first list 
df_headers = header_list[0]

# Remove the first item from our headers list, it is the index header that we do not need
df_headers.remove('Rk')
df_headers.append("Year")

# Set column headers equal to our list
nba_stats_df.columns = df_headers

In [6]:
# Change types of columns to numeric for columns with number values
nba_stats_df[['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']] = nba_stats_df[['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']].apply(pd.to_numeric)

# Drop all players with NaN games to remove null values
nba_stats_df.dropna(subset=['G'], axis = 0 , inplace= True)

# Remove any players with fewer than 27 games played (1/3rd of the season)
filtered_nba_stats_df = nba_stats_df[nba_stats_df['G'] >= 27]

# points, rebounds, assists, steals, blocks, threes, field-goal percentage, and free-throw percentage

# Select the columns we want for our NBA player analysis
yahoo_nba_df = filtered_nba_stats_df[['Year','Player','Age','Pos','Tm','G','FG%','3P','FT%','TRB','AST','STL','BLK','PTS']]


In [7]:
# Sort by index to prepare to drop duplicates
yahoo_nba_df = yahoo_nba_df.sort_index()

# Drop duplicate entries of Player Name and Year
# This is to eliminate partial season data for players who played for 2+ teams in one season
yahoo_nba_df = yahoo_nba_df.drop_duplicates(subset=['Year', 'Player'])

In [8]:
# Sort data by name alphabetically, then by year in descending order
final_yahoo_nba_df = yahoo_nba_df.sort_values(['Year','Player'], ascending=[True, True])
final_yahoo_nba_df

,Year,Player,Age,Pos,Tm,G,FG%,3P,FT%,TRB,AST,STL,BLK,PTS
2955,2018,Aaron Brooks,33,PG,MIN,32,0.406,11,0.727,17,20,6,0,75
3092,2018,Aaron Gordon,22,PF,ORL,58,0.434,115,0.698,457,136,59,45,1022
3323,2018,Abdel Nader,24,SF,BOS,48,0.336,23,0.590,71,26,15,10,146
3155,2018,Al Horford,31,C,BOS,72,0.489,97,0.783,530,339,43,78,927
3184,2018,Al Jefferson,33,C,IND,36,0.534,0,0.833,143,30,16,23,252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766,2022,Yuta Watanabe,27,SF,TOR,38,0.406,27,0.600,92,21,11,16,163
136,2022,Zach Collins,24,C,SAS,28,0.490,14,0.800,153,61,13,23,218
442,2022,Zach LaVine,26,SG,CHI,67,0.476,185,0.853,308,303,41,23,1635
553,2022,Zeke Nnaji,21,PF,DEN,41,0.516,37,0.631,147,17,16,13,270


In [9]:
# Create a new dataframe for stats percentile calculations
percentile_df = pd.DataFrame(columns = ['Year', 'Player', 'Age', 'Pos', 'Tm', '3P_Percentile', 'FG%_Percentile', 'TRB_Percentile', 'FT%_Percentile', 'AST_Percentile', 'STL_Percentile', 'BLK_Percentile', 'PTS_Percentile'])

# Carry over your non-numeric columnns from final_yahoo_nba_df
percentile_df['Year'] = final_yahoo_nba_df['Year']
percentile_df['Player'] = final_yahoo_nba_df['Player']
percentile_df['Age'] = final_yahoo_nba_df['Age']
percentile_df['Pos'] = final_yahoo_nba_df['Pos']
percentile_df['Tm'] = final_yahoo_nba_df['Tm']

# Calculate the percentile rank for each player in each season, seperately, then add all the seasons in one dataframe
for year in last_five_years:
    year_df = final_yahoo_nba_df.loc[final_yahoo_nba_df['Year'] == year]
    year_df['3P_Percentile'] = year_df['3P'].rank(pct=True)
    year_df['FG%_Percentile'] = year_df['FG%'].rank(pct=True)
    year_df['TRB_Percentile'] = year_df['TRB'].rank(pct=True)
    year_df['FT%_Percentile'] = year_df['FT%'].rank(pct=True)
    year_df['AST_Percentile'] = year_df['AST'].rank(pct=True)
    year_df['STL_Percentile'] = year_df['STL'].rank(pct=True)
    year_df['BLK_Percentile'] = year_df['BLK'].rank(pct=True)
    year_df['PTS_Percentile'] = year_df['PTS'].rank(pct=True)
    year_df.sort_values('Player', ascending=True)

    # Each of the seasons are added back to the percentile dataframe
    percentile_df = percentile_df.append(year_df, ignore_index=True)



/var/folders/2d/w0_y9zt547v2y0371_wclbcm0000gn/T/ipykernel_7359/113467155.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year_df['3P_Percentile'] = year_df['3P'].rank(pct=True)
/var/folders/2d/w0_y9zt547v2y0371_wclbcm0000gn/T/ipykernel_7359/113467155.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year_df['FG%_Percentile'] = year_df['FG%'].rank(pct=True)
/var/folders/2d/w0_y9zt547v2y0371_wclbcm0000gn/T/ipykernel_7359/113467155.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [10]:
# Clean up the percentile dataframe, drop NaNs and remove unnecessary columns
percentile_df = percentile_df.dropna()
percentile_df = percentile_df.drop(['Tm','G','FG%','3P','FT%','TRB','AST','STL','BLK','PTS'], axis = 1)

# Add a rank column that adds the percentiles from each category
# Average out percentile categories for FG% and FT% with PTS, as we might still want a player with somewhat  
## lower FG% and FT% if they score a lot of points
percentile_df['Rank'] = (percentile_df['3P_Percentile'] + ((percentile_df['FG%_Percentile'] + percentile_df['PTS_Percentile']) / 2) + percentile_df['TRB_Percentile'] + ((percentile_df['FT%_Percentile'] + percentile_df['PTS_Percentile']) / 2) + percentile_df['AST_Percentile'] + percentile_df['STL_Percentile'] + percentile_df['BLK_Percentile'] + percentile_df['PTS_Percentile'])


In [11]:
# Create a list of each unique player we have in our dataframe
player_list = percentile_df.Player.unique().tolist()

# Create a new dataframe for combined, averaged percentiles over the past 5 seasons
new_df = pd.DataFrame(columns = ['Player', 'Rank', 'Trend', 'Pos', 'Years', '3P_Percentile', 'FG%_Percentile', 'TRB_Percentile', 'FT%_Percentile', 'AST_Percentile', 'STL_Percentile', 'BLK_Percentile', 'PTS_Percentile'])

# Update new dataframe with unique player list
new_df['Player'] = player_list

# Create a list for each percentile stat category for upcoming loop
player_trends = []
average_3P = []
average_FG = []
average_TRB = []
average_FT = []
average_AST = []
average_STL = []
average_BLK = []
average_PTS = []
average_Rank = []
year_count = []
pos = []

# Loop through each player, locate their percentile stats for each season, average them out
for player in player_list:
    player_df = percentile_df.loc[percentile_df['Player'] == player]
    
    # We want to find the slope of the line of best fit for each player's overall ranking each season
    x = np.array(player_df['Year'], dtype = float)
    y = np.array(player_df['Rank'], dtype = float)
    slope, intercept = np.polyfit(x, y, 1)
    player_trends.append(slope)
    
    # Find average of each player's percentiles from previous 5 seasons
    average_3P.append(sum(player_df['3P_Percentile']) / len(player_df['3P_Percentile']))
    average_FG.append(sum(player_df['FG%_Percentile']) / len(player_df['FG%_Percentile']))
    average_TRB.append(sum(player_df['TRB_Percentile']) / len(player_df['TRB_Percentile']))
    average_FT.append(sum(player_df['FT%_Percentile']) / len(player_df['FT%_Percentile']))
    average_AST.append(sum(player_df['AST_Percentile']) / len(player_df['AST_Percentile']))
    average_STL.append(sum(player_df['STL_Percentile']) / len(player_df['STL_Percentile']))
    average_BLK.append(sum(player_df['BLK_Percentile']) / len(player_df['BLK_Percentile']))
    average_PTS.append(sum(player_df['PTS_Percentile']) / len(player_df['PTS_Percentile']))
    average_Rank.append(sum(player_df['Rank']) / len(player_df['Rank']))
    year_count.append(len(x))
    
    # Keep player positions for reference purposes during the draft
    pos.append(player_df['Pos'].unique())

# Update new dataframe with the list data from each stat
new_df['Pos'] = pos
new_df['Trend'] = player_trends
new_df['3P_Percentile'] = average_3P
new_df['FG%_Percentile'] = average_FG
new_df['TRB_Percentile'] = average_TRB
new_df['FT%_Percentile'] = average_FT
new_df['AST_Percentile'] = average_AST
new_df['STL_Percentile'] = average_STL
new_df['BLK_Percentile'] = average_BLK
new_df['PTS_Percentile'] = average_PTS
new_df['Rank'] = average_Rank

# Keep track of how many seasons are being considered, so we know how reliable the data is
new_df['Years'] = year_count



/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user

/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user

/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user

In [12]:
# Create a weighted rank column by adding trend data to the rank data and account for number of seasons played
# Basically, if you played all 5 seasons, your trend stat is added directly
# If you played fewer than all 5 seasons, your trend stat is reduced depending on how few seasons you played
new_df['Weighted Rank'] = (new_df['Rank'] + ((new_df['Trend'] * (new_df['Years'] - 1) / 4)))

# shift column 'Weighted Rank' to first position
first_column = new_df.pop('Weighted Rank')
  
# insert column using insert(position,column_name,first_column) function
new_df.insert(1, 'Weighted Rank', first_column)

new_df = new_df.sort_values('Weighted Rank', ascending = False)
new_df.head(50)

,Player,Weighted Rank,Rank,Trend,Pos,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile
312,Nikola Jokić,7.293490,7.229375,0.064116,[C],5,0.707108,0.850647,0.987655,0.748442,0.985404,0.948232,0.872614,0.964408
191,Jayson Tatum,7.099159,6.857182,0.241977,"[SF, PF]",5,0.892891,0.564035,0.898168,0.783188,0.777362,0.903351,0.844353,0.933723
180,James Harden,7.000622,7.152452,-0.151829,"[PG, PG-SG, SG]",5,0.939523,0.460049,0.864163,0.885841,0.989090,0.933528,0.825727,0.963738
143,Giannis Antetokounmpo,6.914292,6.904490,0.009802,[PF],5,0.565308,0.888105,0.983345,0.292890,0.941954,0.887239,0.959300,0.988424
272,Luka Dončić,6.911124,6.762899,0.197633,"[PG, SG]",4,0.955300,0.556933,0.943542,0.365488,0.976297,0.854443,0.623516,0.974295
313,Nikola Vučević,6.865272,6.701964,0.163308,[C],5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915
228,Karl-Anthony Towns,6.826070,6.845916,-0.019847,[C],5,0.840424,0.818866,0.950154,0.747798,0.802015,0.675441,0.930512,0.932019
455,Kawhi Leonard,6.804561,6.780120,0.048883,[SF],3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323
365,Stephen Curry,6.791723,6.705942,0.114374,[PG],4,0.994109,0.630533,0.734059,0.980854,0.932163,0.907780,0.394058,0.969041
17,Anthony Edwards,6.758378,6.685627,0.291006,[SG],2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205


In [13]:
# new_df.to_csv("/Users/michaelbinger/Documents/Projects/Fantasy-Baseball-Analysis/Fantasy_Basketball/nba_trends.csv")
# final_yahoo_nba_df.to_csv("/Users/michaelbinger/Documents/Projects/Fantasy-Baseball-Analysis/Fantasy_Basketball/nba_stats.csv")


In [14]:
new_pos_list = []
pos_list = new_df['Pos'].tolist()
for item in pos_list:
    new_string = []
    for pos in item:
        string = str(pos)
        new_string = f'{new_string},{string}'
    new_pos_list.append(new_string)

final_pos_list = []
for i in new_pos_list:
    i = i.replace('[],', '')
    final_pos_list.append(i)
    
final_pos_list

new_df.drop('Pos', axis = 1, inplace = True)
new_df['Pos'] = final_pos_list

new_df

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
312,Nikola Jokić,7.293490,7.229375,0.064116,5,0.707108,0.850647,0.987655,0.748442,0.985404,0.948232,0.872614,0.964408,C
191,Jayson Tatum,7.099159,6.857182,0.241977,5,0.892891,0.564035,0.898168,0.783188,0.777362,0.903351,0.844353,0.933723,"SF,PF"
180,James Harden,7.000622,7.152452,-0.151829,5,0.939523,0.460049,0.864163,0.885841,0.989090,0.933528,0.825727,0.963738,"PG,PG-SG,SG"
143,Giannis Antetokounmpo,6.914292,6.904490,0.009802,5,0.565308,0.888105,0.983345,0.292890,0.941954,0.887239,0.959300,0.988424,PF
272,Luka Dončić,6.911124,6.762899,0.197633,4,0.955300,0.556933,0.943542,0.365488,0.976297,0.854443,0.623516,0.974295,"PG,SG"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,Keljin Blevins,0.536659,0.536659,0.000133,1,0.242788,0.009615,0.028846,0.042067,0.049279,0.106971,0.020433,0.031250,SF
453,Juwan Morgan,0.501235,0.501235,0.000124,1,0.111111,0.590123,0.014815,0.002469,0.017284,0.018519,0.038272,0.002469,PF
661,Marcus Georges-Hunt,0.480263,0.480263,0.000119,1,0.164474,0.409211,0.003947,0.088158,0.005263,0.032895,0.009211,0.007895,SG
523,Malcolm Miller,0.325858,0.325858,0.000081,1,0.149077,0.241425,0.002639,0.006596,0.007916,0.011873,0.025066,0.002639,SF


In [15]:
draft_df = new_df

In [16]:
######################################################################################################################
######################################################################################################################
######################################################################################################################
#### DRAFT DAY FUNCTIONS
   
# DROP A PLAYER 
def drafted(player):
    global draft_df
    draft_df = draft_df[draft_df.Player != player]
    return draft_df.head(25)
    
# FILTER PLAYERS BY POSITION
def position_filter(Pos):
    filtered_draft_df = draft_df[draft_df['Pos'].str.contains(Pos)]
    return filtered_draft_df.head(25)

# PULL STAT CATEGORY LEADERS
def stat_leaders(CAT):
    global final_yahoo_nba_df
    final_yahoo_nba_df = final_yahoo_nba_df.sort_values([CAT], ascending=[False])
    return final_yahoo_nba_df.head(25)

In [17]:
# draft_df = draft_df.sort_values(['Weighted Rank'], ascending=[False])
# draft_df.head(50)

In [18]:
drafted('Anthony Davis')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
312,Nikola Jokić,7.293490,7.229375,0.064116,5,0.707108,0.850647,0.987655,0.748442,0.985404,0.948232,0.872614,0.964408,C
191,Jayson Tatum,7.099159,6.857182,0.241977,5,0.892891,0.564035,0.898168,0.783188,0.777362,0.903351,0.844353,0.933723,"SF,PF"
180,James Harden,7.000622,7.152452,-0.151829,5,0.939523,0.460049,0.864163,0.885841,0.989090,0.933528,0.825727,0.963738,"PG,PG-SG,SG"
143,Giannis Antetokounmpo,6.914292,6.904490,0.009802,5,0.565308,0.888105,0.983345,0.292890,0.941954,0.887239,0.959300,0.988424,PF
272,Luka Dončić,6.911124,6.762899,0.197633,4,0.955300,0.556933,0.943542,0.365488,0.976297,0.854443,0.623516,0.974295,"PG,SG"
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
228,Karl-Anthony Towns,6.826070,6.845916,-0.019847,5,0.840424,0.818866,0.950154,0.747798,0.802015,0.675441,0.930512,0.932019,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
365,Stephen Curry,6.791723,6.705942,0.114374,4,0.994109,0.630533,0.734059,0.980854,0.932163,0.907780,0.394058,0.969041,PG
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG


In [19]:
drafted('Kevin Durant')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
312,Nikola Jokić,7.293490,7.229375,0.064116,5,0.707108,0.850647,0.987655,0.748442,0.985404,0.948232,0.872614,0.964408,C
191,Jayson Tatum,7.099159,6.857182,0.241977,5,0.892891,0.564035,0.898168,0.783188,0.777362,0.903351,0.844353,0.933723,"SF,PF"
180,James Harden,7.000622,7.152452,-0.151829,5,0.939523,0.460049,0.864163,0.885841,0.989090,0.933528,0.825727,0.963738,"PG,PG-SG,SG"
143,Giannis Antetokounmpo,6.914292,6.904490,0.009802,5,0.565308,0.888105,0.983345,0.292890,0.941954,0.887239,0.959300,0.988424,PF
272,Luka Dončić,6.911124,6.762899,0.197633,4,0.955300,0.556933,0.943542,0.365488,0.976297,0.854443,0.623516,0.974295,"PG,SG"
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
228,Karl-Anthony Towns,6.826070,6.845916,-0.019847,5,0.840424,0.818866,0.950154,0.747798,0.802015,0.675441,0.930512,0.932019,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
365,Stephen Curry,6.791723,6.705942,0.114374,4,0.994109,0.630533,0.734059,0.980854,0.932163,0.907780,0.394058,0.969041,PG
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG


In [20]:
drafted('Devin Booker')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
312,Nikola Jokić,7.293490,7.229375,0.064116,5,0.707108,0.850647,0.987655,0.748442,0.985404,0.948232,0.872614,0.964408,C
191,Jayson Tatum,7.099159,6.857182,0.241977,5,0.892891,0.564035,0.898168,0.783188,0.777362,0.903351,0.844353,0.933723,"SF,PF"
180,James Harden,7.000622,7.152452,-0.151829,5,0.939523,0.460049,0.864163,0.885841,0.989090,0.933528,0.825727,0.963738,"PG,PG-SG,SG"
143,Giannis Antetokounmpo,6.914292,6.904490,0.009802,5,0.565308,0.888105,0.983345,0.292890,0.941954,0.887239,0.959300,0.988424,PF
272,Luka Dončić,6.911124,6.762899,0.197633,4,0.955300,0.556933,0.943542,0.365488,0.976297,0.854443,0.623516,0.974295,"PG,SG"
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
228,Karl-Anthony Towns,6.826070,6.845916,-0.019847,5,0.840424,0.818866,0.950154,0.747798,0.802015,0.675441,0.930512,0.932019,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
365,Stephen Curry,6.791723,6.705942,0.114374,4,0.994109,0.630533,0.734059,0.980854,0.932163,0.907780,0.394058,0.969041,PG
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG


In [21]:
drafted('Lebron James')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
312,Nikola Jokić,7.293490,7.229375,0.064116,5,0.707108,0.850647,0.987655,0.748442,0.985404,0.948232,0.872614,0.964408,C
191,Jayson Tatum,7.099159,6.857182,0.241977,5,0.892891,0.564035,0.898168,0.783188,0.777362,0.903351,0.844353,0.933723,"SF,PF"
180,James Harden,7.000622,7.152452,-0.151829,5,0.939523,0.460049,0.864163,0.885841,0.989090,0.933528,0.825727,0.963738,"PG,PG-SG,SG"
143,Giannis Antetokounmpo,6.914292,6.904490,0.009802,5,0.565308,0.888105,0.983345,0.292890,0.941954,0.887239,0.959300,0.988424,PF
272,Luka Dončić,6.911124,6.762899,0.197633,4,0.955300,0.556933,0.943542,0.365488,0.976297,0.854443,0.623516,0.974295,"PG,SG"
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
228,Karl-Anthony Towns,6.826070,6.845916,-0.019847,5,0.840424,0.818866,0.950154,0.747798,0.802015,0.675441,0.930512,0.932019,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
365,Stephen Curry,6.791723,6.705942,0.114374,4,0.994109,0.630533,0.734059,0.980854,0.932163,0.907780,0.394058,0.969041,PG
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG


In [22]:
drafted('Nikola Jokić')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
191,Jayson Tatum,7.099159,6.857182,0.241977,5,0.892891,0.564035,0.898168,0.783188,0.777362,0.903351,0.844353,0.933723,"SF,PF"
180,James Harden,7.000622,7.152452,-0.151829,5,0.939523,0.460049,0.864163,0.885841,0.989090,0.933528,0.825727,0.963738,"PG,PG-SG,SG"
143,Giannis Antetokounmpo,6.914292,6.904490,0.009802,5,0.565308,0.888105,0.983345,0.292890,0.941954,0.887239,0.959300,0.988424,PF
272,Luka Dončić,6.911124,6.762899,0.197633,4,0.955300,0.556933,0.943542,0.365488,0.976297,0.854443,0.623516,0.974295,"PG,SG"
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
228,Karl-Anthony Towns,6.826070,6.845916,-0.019847,5,0.840424,0.818866,0.950154,0.747798,0.802015,0.675441,0.930512,0.932019,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
365,Stephen Curry,6.791723,6.705942,0.114374,4,0.994109,0.630533,0.734059,0.980854,0.932163,0.907780,0.394058,0.969041,PG
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
260,LaMelo Ball,6.753342,6.420747,1.330382,2,0.841750,0.372215,0.833989,0.655757,0.950846,0.964377,0.613324,0.851238,PG


In [23]:
drafted('Ja Morant')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
191,Jayson Tatum,7.099159,6.857182,0.241977,5,0.892891,0.564035,0.898168,0.783188,0.777362,0.903351,0.844353,0.933723,"SF,PF"
180,James Harden,7.000622,7.152452,-0.151829,5,0.939523,0.460049,0.864163,0.885841,0.989090,0.933528,0.825727,0.963738,"PG,PG-SG,SG"
143,Giannis Antetokounmpo,6.914292,6.904490,0.009802,5,0.565308,0.888105,0.983345,0.292890,0.941954,0.887239,0.959300,0.988424,PF
272,Luka Dončić,6.911124,6.762899,0.197633,4,0.955300,0.556933,0.943542,0.365488,0.976297,0.854443,0.623516,0.974295,"PG,SG"
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
228,Karl-Anthony Towns,6.826070,6.845916,-0.019847,5,0.840424,0.818866,0.950154,0.747798,0.802015,0.675441,0.930512,0.932019,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
365,Stephen Curry,6.791723,6.705942,0.114374,4,0.994109,0.630533,0.734059,0.980854,0.932163,0.907780,0.394058,0.969041,PG
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
260,LaMelo Ball,6.753342,6.420747,1.330382,2,0.841750,0.372215,0.833989,0.655757,0.950846,0.964377,0.613324,0.851238,PG


In [24]:
drafted('Luka Dončić')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
191,Jayson Tatum,7.099159,6.857182,0.241977,5,0.892891,0.564035,0.898168,0.783188,0.777362,0.903351,0.844353,0.933723,"SF,PF"
180,James Harden,7.000622,7.152452,-0.151829,5,0.939523,0.460049,0.864163,0.885841,0.989090,0.933528,0.825727,0.963738,"PG,PG-SG,SG"
143,Giannis Antetokounmpo,6.914292,6.904490,0.009802,5,0.565308,0.888105,0.983345,0.292890,0.941954,0.887239,0.959300,0.988424,PF
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
228,Karl-Anthony Towns,6.826070,6.845916,-0.019847,5,0.840424,0.818866,0.950154,0.747798,0.802015,0.675441,0.930512,0.932019,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
365,Stephen Curry,6.791723,6.705942,0.114374,4,0.994109,0.630533,0.734059,0.980854,0.932163,0.907780,0.394058,0.969041,PG
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
260,LaMelo Ball,6.753342,6.420747,1.330382,2,0.841750,0.372215,0.833989,0.655757,0.950846,0.964377,0.613324,0.851238,PG
202,Joel Embiid,6.745065,6.570410,0.174656,5,0.602249,0.739343,0.974559,0.647598,0.770662,0.660812,0.961258,0.953700,C


In [25]:
drafted('Joel Embiid')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
191,Jayson Tatum,7.099159,6.857182,0.241977,5,0.892891,0.564035,0.898168,0.783188,0.777362,0.903351,0.844353,0.933723,"SF,PF"
180,James Harden,7.000622,7.152452,-0.151829,5,0.939523,0.460049,0.864163,0.885841,0.989090,0.933528,0.825727,0.963738,"PG,PG-SG,SG"
143,Giannis Antetokounmpo,6.914292,6.904490,0.009802,5,0.565308,0.888105,0.983345,0.292890,0.941954,0.887239,0.959300,0.988424,PF
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
228,Karl-Anthony Towns,6.826070,6.845916,-0.019847,5,0.840424,0.818866,0.950154,0.747798,0.802015,0.675441,0.930512,0.932019,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
365,Stephen Curry,6.791723,6.705942,0.114374,4,0.994109,0.630533,0.734059,0.980854,0.932163,0.907780,0.394058,0.969041,PG
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
260,LaMelo Ball,6.753342,6.420747,1.330382,2,0.841750,0.372215,0.833989,0.655757,0.950846,0.964377,0.613324,0.851238,PG
266,LeBron James,6.739532,6.819356,-0.079824,5,0.849661,0.821348,0.899853,0.287371,0.964418,0.855531,0.784116,0.955708,"PF,PG,SF"


In [26]:
drafted('Russell Westbrook')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
191,Jayson Tatum,7.099159,6.857182,0.241977,5,0.892891,0.564035,0.898168,0.783188,0.777362,0.903351,0.844353,0.933723,"SF,PF"
180,James Harden,7.000622,7.152452,-0.151829,5,0.939523,0.460049,0.864163,0.885841,0.989090,0.933528,0.825727,0.963738,"PG,PG-SG,SG"
143,Giannis Antetokounmpo,6.914292,6.904490,0.009802,5,0.565308,0.888105,0.983345,0.292890,0.941954,0.887239,0.959300,0.988424,PF
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
228,Karl-Anthony Towns,6.826070,6.845916,-0.019847,5,0.840424,0.818866,0.950154,0.747798,0.802015,0.675441,0.930512,0.932019,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
365,Stephen Curry,6.791723,6.705942,0.114374,4,0.994109,0.630533,0.734059,0.980854,0.932163,0.907780,0.394058,0.969041,PG
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
260,LaMelo Ball,6.753342,6.420747,1.330382,2,0.841750,0.372215,0.833989,0.655757,0.950846,0.964377,0.613324,0.851238,PG
266,LeBron James,6.739532,6.819356,-0.079824,5,0.849661,0.821348,0.899853,0.287371,0.964418,0.855531,0.784116,0.955708,"PF,PG,SF"


In [27]:
drafted('Karl-Anthony Towns')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
191,Jayson Tatum,7.099159,6.857182,0.241977,5,0.892891,0.564035,0.898168,0.783188,0.777362,0.903351,0.844353,0.933723,"SF,PF"
180,James Harden,7.000622,7.152452,-0.151829,5,0.939523,0.460049,0.864163,0.885841,0.989090,0.933528,0.825727,0.963738,"PG,PG-SG,SG"
143,Giannis Antetokounmpo,6.914292,6.904490,0.009802,5,0.565308,0.888105,0.983345,0.292890,0.941954,0.887239,0.959300,0.988424,PF
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
365,Stephen Curry,6.791723,6.705942,0.114374,4,0.994109,0.630533,0.734059,0.980854,0.932163,0.907780,0.394058,0.969041,PG
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
260,LaMelo Ball,6.753342,6.420747,1.330382,2,0.841750,0.372215,0.833989,0.655757,0.950846,0.964377,0.613324,0.851238,PG
266,LeBron James,6.739532,6.819356,-0.079824,5,0.849661,0.821348,0.899853,0.287371,0.964418,0.855531,0.784116,0.955708,"PF,PG,SF"
401,Tyrese Haliburton,6.673169,6.438286,0.939534,2,0.868639,0.635062,0.585236,0.782894,0.954419,0.963613,0.773564,0.791919,"SG-PG,PG"


In [28]:
drafted('Kyrie Irving')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
191,Jayson Tatum,7.099159,6.857182,0.241977,5,0.892891,0.564035,0.898168,0.783188,0.777362,0.903351,0.844353,0.933723,"SF,PF"
180,James Harden,7.000622,7.152452,-0.151829,5,0.939523,0.460049,0.864163,0.885841,0.989090,0.933528,0.825727,0.963738,"PG,PG-SG,SG"
143,Giannis Antetokounmpo,6.914292,6.904490,0.009802,5,0.565308,0.888105,0.983345,0.292890,0.941954,0.887239,0.959300,0.988424,PF
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
365,Stephen Curry,6.791723,6.705942,0.114374,4,0.994109,0.630533,0.734059,0.980854,0.932163,0.907780,0.394058,0.969041,PG
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
260,LaMelo Ball,6.753342,6.420747,1.330382,2,0.841750,0.372215,0.833989,0.655757,0.950846,0.964377,0.613324,0.851238,PG
266,LeBron James,6.739532,6.819356,-0.079824,5,0.849661,0.821348,0.899853,0.287371,0.964418,0.855531,0.784116,0.955708,"PF,PG,SF"
401,Tyrese Haliburton,6.673169,6.438286,0.939534,2,0.868639,0.635062,0.585236,0.782894,0.954419,0.963613,0.773564,0.791919,"SG-PG,PG"


In [29]:
drafted('Stephen Curry')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
191,Jayson Tatum,7.099159,6.857182,0.241977,5,0.892891,0.564035,0.898168,0.783188,0.777362,0.903351,0.844353,0.933723,"SF,PF"
180,James Harden,7.000622,7.152452,-0.151829,5,0.939523,0.460049,0.864163,0.885841,0.989090,0.933528,0.825727,0.963738,"PG,PG-SG,SG"
143,Giannis Antetokounmpo,6.914292,6.904490,0.009802,5,0.565308,0.888105,0.983345,0.292890,0.941954,0.887239,0.959300,0.988424,PF
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
260,LaMelo Ball,6.753342,6.420747,1.330382,2,0.841750,0.372215,0.833989,0.655757,0.950846,0.964377,0.613324,0.851238,PG
266,LeBron James,6.739532,6.819356,-0.079824,5,0.849661,0.821348,0.899853,0.287371,0.964418,0.855531,0.784116,0.955708,"PF,PG,SF"
401,Tyrese Haliburton,6.673169,6.438286,0.939534,2,0.868639,0.635062,0.585236,0.782894,0.954419,0.963613,0.773564,0.791919,"SG-PG,PG"
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF


In [30]:
drafted('Rudy Gobert')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
191,Jayson Tatum,7.099159,6.857182,0.241977,5,0.892891,0.564035,0.898168,0.783188,0.777362,0.903351,0.844353,0.933723,"SF,PF"
180,James Harden,7.000622,7.152452,-0.151829,5,0.939523,0.460049,0.864163,0.885841,0.989090,0.933528,0.825727,0.963738,"PG,PG-SG,SG"
143,Giannis Antetokounmpo,6.914292,6.904490,0.009802,5,0.565308,0.888105,0.983345,0.292890,0.941954,0.887239,0.959300,0.988424,PF
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
260,LaMelo Ball,6.753342,6.420747,1.330382,2,0.841750,0.372215,0.833989,0.655757,0.950846,0.964377,0.613324,0.851238,PG
266,LeBron James,6.739532,6.819356,-0.079824,5,0.849661,0.821348,0.899853,0.287371,0.964418,0.855531,0.784116,0.955708,"PF,PG,SF"
401,Tyrese Haliburton,6.673169,6.438286,0.939534,2,0.868639,0.635062,0.585236,0.782894,0.954419,0.963613,0.773564,0.791919,"SG-PG,PG"
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF


In [31]:
drafted('Giannis Antetokounmpo')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
191,Jayson Tatum,7.099159,6.857182,0.241977,5,0.892891,0.564035,0.898168,0.783188,0.777362,0.903351,0.844353,0.933723,"SF,PF"
180,James Harden,7.000622,7.152452,-0.151829,5,0.939523,0.460049,0.864163,0.885841,0.989090,0.933528,0.825727,0.963738,"PG,PG-SG,SG"
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
260,LaMelo Ball,6.753342,6.420747,1.330382,2,0.841750,0.372215,0.833989,0.655757,0.950846,0.964377,0.613324,0.851238,PG
266,LeBron James,6.739532,6.819356,-0.079824,5,0.849661,0.821348,0.899853,0.287371,0.964418,0.855531,0.784116,0.955708,"PF,PG,SF"
401,Tyrese Haliburton,6.673169,6.438286,0.939534,2,0.868639,0.635062,0.585236,0.782894,0.954419,0.963613,0.773564,0.791919,"SG-PG,PG"
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF


In [32]:
drafted('Zion Williamson')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
191,Jayson Tatum,7.099159,6.857182,0.241977,5,0.892891,0.564035,0.898168,0.783188,0.777362,0.903351,0.844353,0.933723,"SF,PF"
180,James Harden,7.000622,7.152452,-0.151829,5,0.939523,0.460049,0.864163,0.885841,0.989090,0.933528,0.825727,0.963738,"PG,PG-SG,SG"
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
260,LaMelo Ball,6.753342,6.420747,1.330382,2,0.841750,0.372215,0.833989,0.655757,0.950846,0.964377,0.613324,0.851238,PG
266,LeBron James,6.739532,6.819356,-0.079824,5,0.849661,0.821348,0.899853,0.287371,0.964418,0.855531,0.784116,0.955708,"PF,PG,SF"
401,Tyrese Haliburton,6.673169,6.438286,0.939534,2,0.868639,0.635062,0.585236,0.782894,0.954419,0.963613,0.773564,0.791919,"SG-PG,PG"
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF


In [33]:
drafted('Damian Lillard')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
191,Jayson Tatum,7.099159,6.857182,0.241977,5,0.892891,0.564035,0.898168,0.783188,0.777362,0.903351,0.844353,0.933723,"SF,PF"
180,James Harden,7.000622,7.152452,-0.151829,5,0.939523,0.460049,0.864163,0.885841,0.989090,0.933528,0.825727,0.963738,"PG,PG-SG,SG"
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
260,LaMelo Ball,6.753342,6.420747,1.330382,2,0.841750,0.372215,0.833989,0.655757,0.950846,0.964377,0.613324,0.851238,PG
266,LeBron James,6.739532,6.819356,-0.079824,5,0.849661,0.821348,0.899853,0.287371,0.964418,0.855531,0.784116,0.955708,"PF,PG,SF"
401,Tyrese Haliburton,6.673169,6.438286,0.939534,2,0.868639,0.635062,0.585236,0.782894,0.954419,0.963613,0.773564,0.791919,"SG-PG,PG"
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF


In [34]:
drafted('Jayson Tatum')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
180,James Harden,7.000622,7.152452,-0.151829,5,0.939523,0.460049,0.864163,0.885841,0.989090,0.933528,0.825727,0.963738,"PG,PG-SG,SG"
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
260,LaMelo Ball,6.753342,6.420747,1.330382,2,0.841750,0.372215,0.833989,0.655757,0.950846,0.964377,0.613324,0.851238,PG
266,LeBron James,6.739532,6.819356,-0.079824,5,0.849661,0.821348,0.899853,0.287371,0.964418,0.855531,0.784116,0.955708,"PF,PG,SF"
401,Tyrese Haliburton,6.673169,6.438286,0.939534,2,0.868639,0.635062,0.585236,0.782894,0.954419,0.963613,0.773564,0.791919,"SG-PG,PG"
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"


In [35]:
drafted('Paul George')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
180,James Harden,7.000622,7.152452,-0.151829,5,0.939523,0.460049,0.864163,0.885841,0.989090,0.933528,0.825727,0.963738,"PG,PG-SG,SG"
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
260,LaMelo Ball,6.753342,6.420747,1.330382,2,0.841750,0.372215,0.833989,0.655757,0.950846,0.964377,0.613324,0.851238,PG
266,LeBron James,6.739532,6.819356,-0.079824,5,0.849661,0.821348,0.899853,0.287371,0.964418,0.855531,0.784116,0.955708,"PF,PG,SF"
401,Tyrese Haliburton,6.673169,6.438286,0.939534,2,0.868639,0.635062,0.585236,0.782894,0.954419,0.963613,0.773564,0.791919,"SG-PG,PG"
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"


In [36]:
drafted('Trae Young')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
180,James Harden,7.000622,7.152452,-0.151829,5,0.939523,0.460049,0.864163,0.885841,0.989090,0.933528,0.825727,0.963738,"PG,PG-SG,SG"
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
260,LaMelo Ball,6.753342,6.420747,1.330382,2,0.841750,0.372215,0.833989,0.655757,0.950846,0.964377,0.613324,0.851238,PG
266,LeBron James,6.739532,6.819356,-0.079824,5,0.849661,0.821348,0.899853,0.287371,0.964418,0.855531,0.784116,0.955708,"PF,PG,SF"
401,Tyrese Haliburton,6.673169,6.438286,0.939534,2,0.868639,0.635062,0.585236,0.782894,0.954419,0.963613,0.773564,0.791919,"SG-PG,PG"
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"


In [37]:
drafted('James Harden')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
260,LaMelo Ball,6.753342,6.420747,1.330382,2,0.841750,0.372215,0.833989,0.655757,0.950846,0.964377,0.613324,0.851238,PG
266,LeBron James,6.739532,6.819356,-0.079824,5,0.849661,0.821348,0.899853,0.287371,0.964418,0.855531,0.784116,0.955708,"PF,PG,SF"
401,Tyrese Haliburton,6.673169,6.438286,0.939534,2,0.868639,0.635062,0.585236,0.782894,0.954419,0.963613,0.773564,0.791919,"SG-PG,PG"
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
413,Zach LaVine,6.505798,6.481359,0.032586,4,0.927815,0.658192,0.707752,0.746246,0.874444,0.715442,0.637827,0.957930,"SG,SF"


In [38]:
drafted('Dejounte Murray')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
260,LaMelo Ball,6.753342,6.420747,1.330382,2,0.841750,0.372215,0.833989,0.655757,0.950846,0.964377,0.613324,0.851238,PG
266,LeBron James,6.739532,6.819356,-0.079824,5,0.849661,0.821348,0.899853,0.287371,0.964418,0.855531,0.784116,0.955708,"PF,PG,SF"
401,Tyrese Haliburton,6.673169,6.438286,0.939534,2,0.868639,0.635062,0.585236,0.782894,0.954419,0.963613,0.773564,0.791919,"SG-PG,PG"
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
413,Zach LaVine,6.505798,6.481359,0.032586,4,0.927815,0.658192,0.707752,0.746246,0.874444,0.715442,0.637827,0.957930,"SG,SF"


In [39]:
drafted('Ben Simmons')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
260,LaMelo Ball,6.753342,6.420747,1.330382,2,0.841750,0.372215,0.833989,0.655757,0.950846,0.964377,0.613324,0.851238,PG
266,LeBron James,6.739532,6.819356,-0.079824,5,0.849661,0.821348,0.899853,0.287371,0.964418,0.855531,0.784116,0.955708,"PF,PG,SF"
401,Tyrese Haliburton,6.673169,6.438286,0.939534,2,0.868639,0.635062,0.585236,0.782894,0.954419,0.963613,0.773564,0.791919,"SG-PG,PG"
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
413,Zach LaVine,6.505798,6.481359,0.032586,4,0.927815,0.658192,0.707752,0.746246,0.874444,0.715442,0.637827,0.957930,"SG,SF"


In [40]:
drafted('LaMelo Ball')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
266,LeBron James,6.739532,6.819356,-0.079824,5,0.849661,0.821348,0.899853,0.287371,0.964418,0.855531,0.784116,0.955708,"PF,PG,SF"
401,Tyrese Haliburton,6.673169,6.438286,0.939534,2,0.868639,0.635062,0.585236,0.782894,0.954419,0.963613,0.773564,0.791919,"SG-PG,PG"
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
413,Zach LaVine,6.505798,6.481359,0.032586,4,0.927815,0.658192,0.707752,0.746246,0.874444,0.715442,0.637827,0.957930,"SG,SF"
293,Miles Bridges,6.502122,5.951252,0.734494,4,0.747779,0.606524,0.828314,0.631800,0.661266,0.684134,0.878504,0.766046,"PF,SF"


In [41]:
drafted('Josh Giddey')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
266,LeBron James,6.739532,6.819356,-0.079824,5,0.849661,0.821348,0.899853,0.287371,0.964418,0.855531,0.784116,0.955708,"PF,PG,SF"
401,Tyrese Haliburton,6.673169,6.438286,0.939534,2,0.868639,0.635062,0.585236,0.782894,0.954419,0.963613,0.773564,0.791919,"SG-PG,PG"
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
413,Zach LaVine,6.505798,6.481359,0.032586,4,0.927815,0.658192,0.707752,0.746246,0.874444,0.715442,0.637827,0.957930,"SG,SF"
293,Miles Bridges,6.502122,5.951252,0.734494,4,0.747779,0.606524,0.828314,0.631800,0.661266,0.684134,0.878504,0.766046,"PF,SF"


In [42]:
drafted('Deandre Ayton')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
266,LeBron James,6.739532,6.819356,-0.079824,5,0.849661,0.821348,0.899853,0.287371,0.964418,0.855531,0.784116,0.955708,"PF,PG,SF"
401,Tyrese Haliburton,6.673169,6.438286,0.939534,2,0.868639,0.635062,0.585236,0.782894,0.954419,0.963613,0.773564,0.791919,"SG-PG,PG"
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
413,Zach LaVine,6.505798,6.481359,0.032586,4,0.927815,0.658192,0.707752,0.746246,0.874444,0.715442,0.637827,0.957930,"SG,SF"
293,Miles Bridges,6.502122,5.951252,0.734494,4,0.747779,0.606524,0.828314,0.631800,0.661266,0.684134,0.878504,0.766046,"PF,SF"


In [43]:
drafted('Steven Adams')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
266,LeBron James,6.739532,6.819356,-0.079824,5,0.849661,0.821348,0.899853,0.287371,0.964418,0.855531,0.784116,0.955708,"PF,PG,SF"
401,Tyrese Haliburton,6.673169,6.438286,0.939534,2,0.868639,0.635062,0.585236,0.782894,0.954419,0.963613,0.773564,0.791919,"SG-PG,PG"
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
413,Zach LaVine,6.505798,6.481359,0.032586,4,0.927815,0.658192,0.707752,0.746246,0.874444,0.715442,0.637827,0.957930,"SG,SF"
293,Miles Bridges,6.502122,5.951252,0.734494,4,0.747779,0.606524,0.828314,0.631800,0.661266,0.684134,0.878504,0.766046,"PF,SF"


In [44]:
drafted('Tyrese Haliburton')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
266,LeBron James,6.739532,6.819356,-0.079824,5,0.849661,0.821348,0.899853,0.287371,0.964418,0.855531,0.784116,0.955708,"PF,PG,SF"
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
413,Zach LaVine,6.505798,6.481359,0.032586,4,0.927815,0.658192,0.707752,0.746246,0.874444,0.715442,0.637827,0.957930,"SG,SF"
293,Miles Bridges,6.502122,5.951252,0.734494,4,0.747779,0.606524,0.828314,0.631800,0.661266,0.684134,0.878504,0.766046,"PF,SF"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG


In [45]:
drafted('LeBron James')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
413,Zach LaVine,6.505798,6.481359,0.032586,4,0.927815,0.658192,0.707752,0.746246,0.874444,0.715442,0.637827,0.957930,"SG,SF"
293,Miles Bridges,6.502122,5.951252,0.734494,4,0.747779,0.606524,0.828314,0.631800,0.661266,0.684134,0.878504,0.766046,"PF,SF"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF


In [46]:
drafted('Shai Gilgeous-Alexander')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
413,Zach LaVine,6.505798,6.481359,0.032586,4,0.927815,0.658192,0.707752,0.746246,0.874444,0.715442,0.637827,0.957930,"SG,SF"
293,Miles Bridges,6.502122,5.951252,0.734494,4,0.747779,0.606524,0.828314,0.631800,0.661266,0.684134,0.878504,0.766046,"PF,SF"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF


In [47]:
drafted('Kawhi Leonard')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
413,Zach LaVine,6.505798,6.481359,0.032586,4,0.927815,0.658192,0.707752,0.746246,0.874444,0.715442,0.637827,0.957930,"SG,SF"
293,Miles Bridges,6.502122,5.951252,0.734494,4,0.747779,0.606524,0.828314,0.631800,0.661266,0.684134,0.878504,0.766046,"PF,SF"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
290,Mikal Bridges,6.470275,6.174145,0.394841,4,0.755662,0.737097,0.715074,0.728974,0.711383,0.960373,0.825551,0.736533,SF


In [48]:
drafted('Fred VanVleet')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
413,Zach LaVine,6.505798,6.481359,0.032586,4,0.927815,0.658192,0.707752,0.746246,0.874444,0.715442,0.637827,0.957930,"SG,SF"
293,Miles Bridges,6.502122,5.951252,0.734494,4,0.747779,0.606524,0.828314,0.631800,0.661266,0.684134,0.878504,0.766046,"PF,SF"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
290,Mikal Bridges,6.470275,6.174145,0.394841,4,0.755662,0.737097,0.715074,0.728974,0.711383,0.960373,0.825551,0.736533,SF


In [49]:
drafted('Zach LaVine')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
293,Miles Bridges,6.502122,5.951252,0.734494,4,0.747779,0.606524,0.828314,0.631800,0.661266,0.684134,0.878504,0.766046,"PF,SF"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
290,Mikal Bridges,6.470275,6.174145,0.394841,4,0.755662,0.737097,0.715074,0.728974,0.711383,0.960373,0.825551,0.736533,SF
43,Cade Cunningham,6.455529,6.455529,0.001596,1,0.768029,0.292067,0.799279,0.780048,0.930288,0.908654,0.825721,0.843750,SG


In [50]:
drafted('Anthony Edwards')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
293,Miles Bridges,6.502122,5.951252,0.734494,4,0.747779,0.606524,0.828314,0.631800,0.661266,0.684134,0.878504,0.766046,"PF,SF"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
290,Mikal Bridges,6.470275,6.174145,0.394841,4,0.755662,0.737097,0.715074,0.728974,0.711383,0.960373,0.825551,0.736533,SF
43,Cade Cunningham,6.455529,6.455529,0.001596,1,0.768029,0.292067,0.799279,0.780048,0.930288,0.908654,0.825721,0.843750,SG
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"


In [51]:
drafted('Donovan Mitchell')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
293,Miles Bridges,6.502122,5.951252,0.734494,4,0.747779,0.606524,0.828314,0.631800,0.661266,0.684134,0.878504,0.766046,"PF,SF"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
290,Mikal Bridges,6.470275,6.174145,0.394841,4,0.755662,0.737097,0.715074,0.728974,0.711383,0.960373,0.825551,0.736533,SF
43,Cade Cunningham,6.455529,6.455529,0.001596,1,0.768029,0.292067,0.799279,0.780048,0.930288,0.908654,0.825721,0.843750,SG
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"


In [52]:
drafted('Domantas Sabonis')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
293,Miles Bridges,6.502122,5.951252,0.734494,4,0.747779,0.606524,0.828314,0.631800,0.661266,0.684134,0.878504,0.766046,"PF,SF"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
290,Mikal Bridges,6.470275,6.174145,0.394841,4,0.755662,0.737097,0.715074,0.728974,0.711383,0.960373,0.825551,0.736533,SF
43,Cade Cunningham,6.455529,6.455529,0.001596,1,0.768029,0.292067,0.799279,0.780048,0.930288,0.908654,0.825721,0.843750,SG
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"


In [53]:
drafted('Jordan Poole')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
293,Miles Bridges,6.502122,5.951252,0.734494,4,0.747779,0.606524,0.828314,0.631800,0.661266,0.684134,0.878504,0.766046,"PF,SF"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
290,Mikal Bridges,6.470275,6.174145,0.394841,4,0.755662,0.737097,0.715074,0.728974,0.711383,0.960373,0.825551,0.736533,SF
43,Cade Cunningham,6.455529,6.455529,0.001596,1,0.768029,0.292067,0.799279,0.780048,0.930288,0.908654,0.825721,0.843750,SG
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"


In [54]:
drafted('Draymond Green')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
293,Miles Bridges,6.502122,5.951252,0.734494,4,0.747779,0.606524,0.828314,0.631800,0.661266,0.684134,0.878504,0.766046,"PF,SF"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
290,Mikal Bridges,6.470275,6.174145,0.394841,4,0.755662,0.737097,0.715074,0.728974,0.711383,0.960373,0.825551,0.736533,SF
43,Cade Cunningham,6.455529,6.455529,0.001596,1,0.768029,0.292067,0.799279,0.780048,0.930288,0.908654,0.825721,0.843750,SG
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"


In [55]:
drafted('Bam Adebayo')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
293,Miles Bridges,6.502122,5.951252,0.734494,4,0.747779,0.606524,0.828314,0.631800,0.661266,0.684134,0.878504,0.766046,"PF,SF"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
290,Mikal Bridges,6.470275,6.174145,0.394841,4,0.755662,0.737097,0.715074,0.728974,0.711383,0.960373,0.825551,0.736533,SF
43,Cade Cunningham,6.455529,6.455529,0.001596,1,0.768029,0.292067,0.799279,0.780048,0.930288,0.908654,0.825721,0.843750,SG
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"


In [56]:
drafted('Robert Williams')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
293,Miles Bridges,6.502122,5.951252,0.734494,4,0.747779,0.606524,0.828314,0.631800,0.661266,0.684134,0.878504,0.766046,"PF,SF"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
290,Mikal Bridges,6.470275,6.174145,0.394841,4,0.755662,0.737097,0.715074,0.728974,0.711383,0.960373,0.825551,0.736533,SF
43,Cade Cunningham,6.455529,6.455529,0.001596,1,0.768029,0.292067,0.799279,0.780048,0.930288,0.908654,0.825721,0.843750,SG
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"


In [57]:
drafted('Bradley Beal')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
293,Miles Bridges,6.502122,5.951252,0.734494,4,0.747779,0.606524,0.828314,0.631800,0.661266,0.684134,0.878504,0.766046,"PF,SF"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
290,Mikal Bridges,6.470275,6.174145,0.394841,4,0.755662,0.737097,0.715074,0.728974,0.711383,0.960373,0.825551,0.736533,SF
43,Cade Cunningham,6.455529,6.455529,0.001596,1,0.768029,0.292067,0.799279,0.780048,0.930288,0.908654,0.825721,0.843750,SG
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"


In [58]:
drafted('Christian Wood')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
293,Miles Bridges,6.502122,5.951252,0.734494,4,0.747779,0.606524,0.828314,0.631800,0.661266,0.684134,0.878504,0.766046,"PF,SF"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
290,Mikal Bridges,6.470275,6.174145,0.394841,4,0.755662,0.737097,0.715074,0.728974,0.711383,0.960373,0.825551,0.736533,SF
43,Cade Cunningham,6.455529,6.455529,0.001596,1,0.768029,0.292067,0.799279,0.780048,0.930288,0.908654,0.825721,0.843750,SG
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"


In [59]:
drafted('Mikal Bridges')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
293,Miles Bridges,6.502122,5.951252,0.734494,4,0.747779,0.606524,0.828314,0.631800,0.661266,0.684134,0.878504,0.766046,"PF,SF"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
43,Cade Cunningham,6.455529,6.455529,0.001596,1,0.768029,0.292067,0.799279,0.780048,0.930288,0.908654,0.825721,0.843750,SG
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
376,Terry Rozier,6.412996,6.027436,0.385561,5,0.923992,0.299424,0.719120,0.691279,0.857199,0.872610,0.510180,0.824492,"SG,PG"


In [60]:
drafted('DeMar DeRozan')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
293,Miles Bridges,6.502122,5.951252,0.734494,4,0.747779,0.606524,0.828314,0.631800,0.661266,0.684134,0.878504,0.766046,"PF,SF"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
43,Cade Cunningham,6.455529,6.455529,0.001596,1,0.768029,0.292067,0.799279,0.780048,0.930288,0.908654,0.825721,0.843750,SG
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
376,Terry Rozier,6.412996,6.027436,0.385561,5,0.923992,0.299424,0.719120,0.691279,0.857199,0.872610,0.510180,0.824492,"SG,PG"


In [61]:
drafted('Cade Cunningham')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
293,Miles Bridges,6.502122,5.951252,0.734494,4,0.747779,0.606524,0.828314,0.631800,0.661266,0.684134,0.878504,0.766046,"PF,SF"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
376,Terry Rozier,6.412996,6.027436,0.385561,5,0.923992,0.299424,0.719120,0.691279,0.857199,0.872610,0.510180,0.824492,"SG,PG"
131,Franz Wagner,6.375000,6.375000,0.001576,1,0.706731,0.623798,0.805288,0.847356,0.816106,0.824519,0.746394,0.870192,SF


In [62]:
drafted('Miles Bridges')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
376,Terry Rozier,6.412996,6.027436,0.385561,5,0.923992,0.299424,0.719120,0.691279,0.857199,0.872610,0.510180,0.824492,"SG,PG"
131,Franz Wagner,6.375000,6.375000,0.001576,1,0.706731,0.623798,0.805288,0.847356,0.816106,0.824519,0.746394,0.870192,SF
56,Chris Paul,6.345855,6.308612,0.037243,5,0.762440,0.626566,0.701557,0.912412,0.982458,0.971264,0.434718,0.843343,PG


In [63]:
drafted('John Collins')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
376,Terry Rozier,6.412996,6.027436,0.385561,5,0.923992,0.299424,0.719120,0.691279,0.857199,0.872610,0.510180,0.824492,"SG,PG"
131,Franz Wagner,6.375000,6.375000,0.001576,1,0.706731,0.623798,0.805288,0.847356,0.816106,0.824519,0.746394,0.870192,SF
56,Chris Paul,6.345855,6.308612,0.037243,5,0.762440,0.626566,0.701557,0.912412,0.982458,0.971264,0.434718,0.843343,PG


In [64]:
drafted('Jaylen Brown')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
376,Terry Rozier,6.412996,6.027436,0.385561,5,0.923992,0.299424,0.719120,0.691279,0.857199,0.872610,0.510180,0.824492,"SG,PG"
131,Franz Wagner,6.375000,6.375000,0.001576,1,0.706731,0.623798,0.805288,0.847356,0.816106,0.824519,0.746394,0.870192,SF
56,Chris Paul,6.345855,6.308612,0.037243,5,0.762440,0.626566,0.701557,0.912412,0.982458,0.971264,0.434718,0.843343,PG


In [65]:
drafted('Myles Turner')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
376,Terry Rozier,6.412996,6.027436,0.385561,5,0.923992,0.299424,0.719120,0.691279,0.857199,0.872610,0.510180,0.824492,"SG,PG"
131,Franz Wagner,6.375000,6.375000,0.001576,1,0.706731,0.623798,0.805288,0.847356,0.816106,0.824519,0.746394,0.870192,SF
56,Chris Paul,6.345855,6.308612,0.037243,5,0.762440,0.626566,0.701557,0.912412,0.982458,0.971264,0.434718,0.843343,PG


In [66]:
drafted('Jimmy Butler')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
376,Terry Rozier,6.412996,6.027436,0.385561,5,0.923992,0.299424,0.719120,0.691279,0.857199,0.872610,0.510180,0.824492,"SG,PG"
131,Franz Wagner,6.375000,6.375000,0.001576,1,0.706731,0.623798,0.805288,0.847356,0.816106,0.824519,0.746394,0.870192,SF
56,Chris Paul,6.345855,6.308612,0.037243,5,0.762440,0.626566,0.701557,0.912412,0.982458,0.971264,0.434718,0.843343,PG


In [67]:
drafted('Nikola Vučević')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
376,Terry Rozier,6.412996,6.027436,0.385561,5,0.923992,0.299424,0.719120,0.691279,0.857199,0.872610,0.510180,0.824492,"SG,PG"
131,Franz Wagner,6.375000,6.375000,0.001576,1,0.706731,0.623798,0.805288,0.847356,0.816106,0.824519,0.746394,0.870192,SF
56,Chris Paul,6.345855,6.308612,0.037243,5,0.762440,0.626566,0.701557,0.912412,0.982458,0.971264,0.434718,0.843343,PG
356,Scottie Barnes,6.332332,6.332332,0.001566,1,0.519231,0.728365,0.913462,0.349760,0.846154,0.917067,0.890625,0.853365,PF


In [68]:
drafted('Darius Garland')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
376,Terry Rozier,6.412996,6.027436,0.385561,5,0.923992,0.299424,0.719120,0.691279,0.857199,0.872610,0.510180,0.824492,"SG,PG"
131,Franz Wagner,6.375000,6.375000,0.001576,1,0.706731,0.623798,0.805288,0.847356,0.816106,0.824519,0.746394,0.870192,SF
56,Chris Paul,6.345855,6.308612,0.037243,5,0.762440,0.626566,0.701557,0.912412,0.982458,0.971264,0.434718,0.843343,PG
356,Scottie Barnes,6.332332,6.332332,0.001566,1,0.519231,0.728365,0.913462,0.349760,0.846154,0.917067,0.890625,0.853365,PF


In [69]:
drafted('Kristaps Porziņģis')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
376,Terry Rozier,6.412996,6.027436,0.385561,5,0.923992,0.299424,0.719120,0.691279,0.857199,0.872610,0.510180,0.824492,"SG,PG"
131,Franz Wagner,6.375000,6.375000,0.001576,1,0.706731,0.623798,0.805288,0.847356,0.816106,0.824519,0.746394,0.870192,SF
56,Chris Paul,6.345855,6.308612,0.037243,5,0.762440,0.626566,0.701557,0.912412,0.982458,0.971264,0.434718,0.843343,PG
356,Scottie Barnes,6.332332,6.332332,0.001566,1,0.519231,0.728365,0.913462,0.349760,0.846154,0.917067,0.890625,0.853365,PF


In [70]:
drafted('Franz Wagner')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
220,Jrue Holiday,6.579166,6.682343,-0.103177,5,0.812487,0.701028,0.724391,0.464184,0.966932,0.971662,0.817885,0.903190,"PG,SG"
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
376,Terry Rozier,6.412996,6.027436,0.385561,5,0.923992,0.299424,0.719120,0.691279,0.857199,0.872610,0.510180,0.824492,"SG,PG"
56,Chris Paul,6.345855,6.308612,0.037243,5,0.762440,0.626566,0.701557,0.912412,0.982458,0.971264,0.434718,0.843343,PG
356,Scottie Barnes,6.332332,6.332332,0.001566,1,0.519231,0.728365,0.913462,0.349760,0.846154,0.917067,0.890625,0.853365,PF
100,Desmond Bane,6.303170,5.837710,1.861840,2,0.896590,0.593383,0.640399,0.802959,0.698147,0.766833,0.582655,0.777457,SG


In [71]:
drafted('Jrue Holiday')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
376,Terry Rozier,6.412996,6.027436,0.385561,5,0.923992,0.299424,0.719120,0.691279,0.857199,0.872610,0.510180,0.824492,"SG,PG"
56,Chris Paul,6.345855,6.308612,0.037243,5,0.762440,0.626566,0.701557,0.912412,0.982458,0.971264,0.434718,0.843343,PG
356,Scottie Barnes,6.332332,6.332332,0.001566,1,0.519231,0.728365,0.913462,0.349760,0.846154,0.917067,0.890625,0.853365,PF
100,Desmond Bane,6.303170,5.837710,1.861840,2,0.896590,0.593383,0.640399,0.802959,0.698147,0.766833,0.582655,0.777457,SG
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"


In [72]:
drafted('Chris Paul')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
323,Pascal Siakam,6.615056,6.239572,0.375484,5,0.618682,0.702908,0.877007,0.461174,0.821116,0.831496,0.820062,0.844584,PF
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
376,Terry Rozier,6.412996,6.027436,0.385561,5,0.923992,0.299424,0.719120,0.691279,0.857199,0.872610,0.510180,0.824492,"SG,PG"
356,Scottie Barnes,6.332332,6.332332,0.001566,1,0.519231,0.728365,0.913462,0.349760,0.846154,0.917067,0.890625,0.853365,PF
100,Desmond Bane,6.303170,5.837710,1.861840,2,0.896590,0.593383,0.640399,0.802959,0.698147,0.766833,0.582655,0.777457,SG
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
42,CJ McCollum,6.128784,6.258050,-0.129266,5,0.950984,0.542452,0.632941,0.564512,0.851964,0.742347,0.660084,0.933124,SG


In [73]:
drafted('Pascal Siakam')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
376,Terry Rozier,6.412996,6.027436,0.385561,5,0.923992,0.299424,0.719120,0.691279,0.857199,0.872610,0.510180,0.824492,"SG,PG"
356,Scottie Barnes,6.332332,6.332332,0.001566,1,0.519231,0.728365,0.913462,0.349760,0.846154,0.917067,0.890625,0.853365,PF
100,Desmond Bane,6.303170,5.837710,1.861840,2,0.896590,0.593383,0.640399,0.802959,0.698147,0.766833,0.582655,0.777457,SG
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
42,CJ McCollum,6.128784,6.258050,-0.129266,5,0.950984,0.542452,0.632941,0.564512,0.851964,0.742347,0.660084,0.933124,SG
36,Brandon Ingram,6.094677,5.794918,0.299760,5,0.637010,0.641914,0.723063,0.555710,0.824128,0.547569,0.730274,0.867031,"SF,PF"


In [74]:
drafted('Evan Mobley')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
376,Terry Rozier,6.412996,6.027436,0.385561,5,0.923992,0.299424,0.719120,0.691279,0.857199,0.872610,0.510180,0.824492,"SG,PG"
356,Scottie Barnes,6.332332,6.332332,0.001566,1,0.519231,0.728365,0.913462,0.349760,0.846154,0.917067,0.890625,0.853365,PF
100,Desmond Bane,6.303170,5.837710,1.861840,2,0.896590,0.593383,0.640399,0.802959,0.698147,0.766833,0.582655,0.777457,SG
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
42,CJ McCollum,6.128784,6.258050,-0.129266,5,0.950984,0.542452,0.632941,0.564512,0.851964,0.742347,0.660084,0.933124,SG
36,Brandon Ingram,6.094677,5.794918,0.299760,5,0.637010,0.641914,0.723063,0.555710,0.824128,0.547569,0.730274,0.867031,"SF,PF"


In [75]:
drafted('Desmond Bane')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
376,Terry Rozier,6.412996,6.027436,0.385561,5,0.923992,0.299424,0.719120,0.691279,0.857199,0.872610,0.510180,0.824492,"SG,PG"
356,Scottie Barnes,6.332332,6.332332,0.001566,1,0.519231,0.728365,0.913462,0.349760,0.846154,0.917067,0.890625,0.853365,PF
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
42,CJ McCollum,6.128784,6.258050,-0.129266,5,0.950984,0.542452,0.632941,0.564512,0.851964,0.742347,0.660084,0.933124,SG
36,Brandon Ingram,6.094677,5.794918,0.299760,5,0.637010,0.641914,0.723063,0.555710,0.824128,0.547569,0.730274,0.867031,"SF,PF"
444,Jamal Murray,6.080538,6.227522,-0.195979,4,0.876877,0.551648,0.602974,0.908174,0.880383,0.831888,0.523815,0.890837,PG


In [76]:
drafted('Scottie Barnes')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
376,Terry Rozier,6.412996,6.027436,0.385561,5,0.923992,0.299424,0.719120,0.691279,0.857199,0.872610,0.510180,0.824492,"SG,PG"
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
42,CJ McCollum,6.128784,6.258050,-0.129266,5,0.950984,0.542452,0.632941,0.564512,0.851964,0.742347,0.660084,0.933124,SG
36,Brandon Ingram,6.094677,5.794918,0.299760,5,0.637010,0.641914,0.723063,0.555710,0.824128,0.547569,0.730274,0.867031,"SF,PF"
444,Jamal Murray,6.080538,6.227522,-0.195979,4,0.876877,0.551648,0.602974,0.908174,0.880383,0.831888,0.523815,0.890837,PG
353,Saddiq Bey,6.036772,5.855996,0.723104,2,0.962202,0.159006,0.831847,0.717775,0.673988,0.809771,0.420295,0.859752,SF


In [77]:
drafted('Michael Porter Jr.')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
376,Terry Rozier,6.412996,6.027436,0.385561,5,0.923992,0.299424,0.719120,0.691279,0.857199,0.872610,0.510180,0.824492,"SG,PG"
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
42,CJ McCollum,6.128784,6.258050,-0.129266,5,0.950984,0.542452,0.632941,0.564512,0.851964,0.742347,0.660084,0.933124,SG
36,Brandon Ingram,6.094677,5.794918,0.299760,5,0.637010,0.641914,0.723063,0.555710,0.824128,0.547569,0.730274,0.867031,"SF,PF"
444,Jamal Murray,6.080538,6.227522,-0.195979,4,0.876877,0.551648,0.602974,0.908174,0.880383,0.831888,0.523815,0.890837,PG
353,Saddiq Bey,6.036772,5.855996,0.723104,2,0.962202,0.159006,0.831847,0.717775,0.673988,0.809771,0.420295,0.859752,SF


In [78]:
drafted('Jonas Valančiūnas')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
376,Terry Rozier,6.412996,6.027436,0.385561,5,0.923992,0.299424,0.719120,0.691279,0.857199,0.872610,0.510180,0.824492,"SG,PG"
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
42,CJ McCollum,6.128784,6.258050,-0.129266,5,0.950984,0.542452,0.632941,0.564512,0.851964,0.742347,0.660084,0.933124,SG
36,Brandon Ingram,6.094677,5.794918,0.299760,5,0.637010,0.641914,0.723063,0.555710,0.824128,0.547569,0.730274,0.867031,"SF,PF"
444,Jamal Murray,6.080538,6.227522,-0.195979,4,0.876877,0.551648,0.602974,0.908174,0.880383,0.831888,0.523815,0.890837,PG
353,Saddiq Bey,6.036772,5.855996,0.723104,2,0.962202,0.159006,0.831847,0.717775,0.673988,0.809771,0.420295,0.859752,SF


In [79]:
drafted('Klay Thompson')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
376,Terry Rozier,6.412996,6.027436,0.385561,5,0.923992,0.299424,0.719120,0.691279,0.857199,0.872610,0.510180,0.824492,"SG,PG"
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
42,CJ McCollum,6.128784,6.258050,-0.129266,5,0.950984,0.542452,0.632941,0.564512,0.851964,0.742347,0.660084,0.933124,SG
36,Brandon Ingram,6.094677,5.794918,0.299760,5,0.637010,0.641914,0.723063,0.555710,0.824128,0.547569,0.730274,0.867031,"SF,PF"
444,Jamal Murray,6.080538,6.227522,-0.195979,4,0.876877,0.551648,0.602974,0.908174,0.880383,0.831888,0.523815,0.890837,PG
353,Saddiq Bey,6.036772,5.855996,0.723104,2,0.962202,0.159006,0.831847,0.717775,0.673988,0.809771,0.420295,0.859752,SF


In [80]:
drafted('Brandon Ingram')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
376,Terry Rozier,6.412996,6.027436,0.385561,5,0.923992,0.299424,0.719120,0.691279,0.857199,0.872610,0.510180,0.824492,"SG,PG"
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
42,CJ McCollum,6.128784,6.258050,-0.129266,5,0.950984,0.542452,0.632941,0.564512,0.851964,0.742347,0.660084,0.933124,SG
444,Jamal Murray,6.080538,6.227522,-0.195979,4,0.876877,0.551648,0.602974,0.908174,0.880383,0.831888,0.523815,0.890837,PG
353,Saddiq Bey,6.036772,5.855996,0.723104,2,0.962202,0.159006,0.831847,0.717775,0.673988,0.809771,0.420295,0.859752,SF
83,De'Aaron Fox,6.028064,5.810007,0.218057,5,0.607942,0.569147,0.529701,0.327572,0.938982,0.890982,0.651389,0.871326,PG


In [81]:
drafted('Terry Rozier')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
42,CJ McCollum,6.128784,6.258050,-0.129266,5,0.950984,0.542452,0.632941,0.564512,0.851964,0.742347,0.660084,0.933124,SG
444,Jamal Murray,6.080538,6.227522,-0.195979,4,0.876877,0.551648,0.602974,0.908174,0.880383,0.831888,0.523815,0.890837,PG
353,Saddiq Bey,6.036772,5.855996,0.723104,2,0.962202,0.159006,0.831847,0.717775,0.673988,0.809771,0.420295,0.859752,SF
83,De'Aaron Fox,6.028064,5.810007,0.218057,5,0.607942,0.569147,0.529701,0.327572,0.938982,0.890982,0.651389,0.871326,PG
257,Kyle Lowry,5.937895,6.142709,-0.204814,5,0.916170,0.326628,0.708057,0.831471,0.970866,0.837571,0.531439,0.799778,PG


In [82]:
drafted('Saddiq Bey')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
42,CJ McCollum,6.128784,6.258050,-0.129266,5,0.950984,0.542452,0.632941,0.564512,0.851964,0.742347,0.660084,0.933124,SG
444,Jamal Murray,6.080538,6.227522,-0.195979,4,0.876877,0.551648,0.602974,0.908174,0.880383,0.831888,0.523815,0.890837,PG
83,De'Aaron Fox,6.028064,5.810007,0.218057,5,0.607942,0.569147,0.529701,0.327572,0.938982,0.890982,0.651389,0.871326,PG
257,Kyle Lowry,5.937895,6.142709,-0.204814,5,0.916170,0.326628,0.708057,0.831471,0.970866,0.837571,0.531439,0.799778,PG
171,Jae'Sean Tate,5.909369,5.899867,0.038006,2,0.545709,0.764539,0.856214,0.250162,0.780520,0.909902,0.785393,0.757390,"PF,SF"


In [83]:
drafted('Jarrett Allen')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
250,Khris Middleton,6.476090,6.483455,-0.007366,5,0.912550,0.596357,0.843751,0.920787,0.915984,0.870455,0.314993,0.933575,SF
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
42,CJ McCollum,6.128784,6.258050,-0.129266,5,0.950984,0.542452,0.632941,0.564512,0.851964,0.742347,0.660084,0.933124,SG
444,Jamal Murray,6.080538,6.227522,-0.195979,4,0.876877,0.551648,0.602974,0.908174,0.880383,0.831888,0.523815,0.890837,PG
83,De'Aaron Fox,6.028064,5.810007,0.218057,5,0.607942,0.569147,0.529701,0.327572,0.938982,0.890982,0.651389,0.871326,PG
257,Kyle Lowry,5.937895,6.142709,-0.204814,5,0.916170,0.326628,0.708057,0.831471,0.970866,0.837571,0.531439,0.799778,PG
171,Jae'Sean Tate,5.909369,5.899867,0.038006,2,0.545709,0.764539,0.856214,0.250162,0.780520,0.909902,0.785393,0.757390,"PF,SF"


In [84]:
drafted('Khris Middleton')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
42,CJ McCollum,6.128784,6.258050,-0.129266,5,0.950984,0.542452,0.632941,0.564512,0.851964,0.742347,0.660084,0.933124,SG
444,Jamal Murray,6.080538,6.227522,-0.195979,4,0.876877,0.551648,0.602974,0.908174,0.880383,0.831888,0.523815,0.890837,PG
83,De'Aaron Fox,6.028064,5.810007,0.218057,5,0.607942,0.569147,0.529701,0.327572,0.938982,0.890982,0.651389,0.871326,PG
257,Kyle Lowry,5.937895,6.142709,-0.204814,5,0.916170,0.326628,0.708057,0.831471,0.970866,0.837571,0.531439,0.799778,PG
171,Jae'Sean Tate,5.909369,5.899867,0.038006,2,0.545709,0.764539,0.856214,0.250162,0.780520,0.909902,0.785393,0.757390,"PF,SF"
153,Herbert Jones,5.859375,5.859375,0.001449,1,0.507212,0.664663,0.693510,0.746394,0.699519,0.995192,0.912260,0.673077,PF


In [85]:
drafted('Clint Capela')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
42,CJ McCollum,6.128784,6.258050,-0.129266,5,0.950984,0.542452,0.632941,0.564512,0.851964,0.742347,0.660084,0.933124,SG
444,Jamal Murray,6.080538,6.227522,-0.195979,4,0.876877,0.551648,0.602974,0.908174,0.880383,0.831888,0.523815,0.890837,PG
83,De'Aaron Fox,6.028064,5.810007,0.218057,5,0.607942,0.569147,0.529701,0.327572,0.938982,0.890982,0.651389,0.871326,PG
257,Kyle Lowry,5.937895,6.142709,-0.204814,5,0.916170,0.326628,0.708057,0.831471,0.970866,0.837571,0.531439,0.799778,PG
171,Jae'Sean Tate,5.909369,5.899867,0.038006,2,0.545709,0.764539,0.856214,0.250162,0.780520,0.909902,0.785393,0.757390,"PF,SF"
153,Herbert Jones,5.859375,5.859375,0.001449,1,0.507212,0.664663,0.693510,0.746394,0.699519,0.995192,0.912260,0.673077,PF


In [86]:
drafted('Jerami Grant')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
384,Tobias Harris,6.605160,6.643211,-0.038051,5,0.821777,0.701320,0.904963,0.791793,0.806608,0.707506,0.792666,0.931567,PF
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
42,CJ McCollum,6.128784,6.258050,-0.129266,5,0.950984,0.542452,0.632941,0.564512,0.851964,0.742347,0.660084,0.933124,SG
444,Jamal Murray,6.080538,6.227522,-0.195979,4,0.876877,0.551648,0.602974,0.908174,0.880383,0.831888,0.523815,0.890837,PG
83,De'Aaron Fox,6.028064,5.810007,0.218057,5,0.607942,0.569147,0.529701,0.327572,0.938982,0.890982,0.651389,0.871326,PG
257,Kyle Lowry,5.937895,6.142709,-0.204814,5,0.916170,0.326628,0.708057,0.831471,0.970866,0.837571,0.531439,0.799778,PG
171,Jae'Sean Tate,5.909369,5.899867,0.038006,2,0.545709,0.764539,0.856214,0.250162,0.780520,0.909902,0.785393,0.757390,"PF,SF"
153,Herbert Jones,5.859375,5.859375,0.001449,1,0.507212,0.664663,0.693510,0.746394,0.699519,0.995192,0.912260,0.673077,PF


In [87]:
drafted('Tobias Harris')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
42,CJ McCollum,6.128784,6.258050,-0.129266,5,0.950984,0.542452,0.632941,0.564512,0.851964,0.742347,0.660084,0.933124,SG
444,Jamal Murray,6.080538,6.227522,-0.195979,4,0.876877,0.551648,0.602974,0.908174,0.880383,0.831888,0.523815,0.890837,PG
83,De'Aaron Fox,6.028064,5.810007,0.218057,5,0.607942,0.569147,0.529701,0.327572,0.938982,0.890982,0.651389,0.871326,PG
257,Kyle Lowry,5.937895,6.142709,-0.204814,5,0.916170,0.326628,0.708057,0.831471,0.970866,0.837571,0.531439,0.799778,PG
171,Jae'Sean Tate,5.909369,5.899867,0.038006,2,0.545709,0.764539,0.856214,0.250162,0.780520,0.909902,0.785393,0.757390,"PF,SF"
153,Herbert Jones,5.859375,5.859375,0.001449,1,0.507212,0.664663,0.693510,0.746394,0.699519,0.995192,0.912260,0.673077,PF
207,Jordan Clarkson,5.842953,5.667340,0.175612,5,0.902518,0.445016,0.578957,0.772534,0.749487,0.729532,0.320857,0.888608,SG


In [88]:
drafted('Tyler Herro')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
42,CJ McCollum,6.128784,6.258050,-0.129266,5,0.950984,0.542452,0.632941,0.564512,0.851964,0.742347,0.660084,0.933124,SG
444,Jamal Murray,6.080538,6.227522,-0.195979,4,0.876877,0.551648,0.602974,0.908174,0.880383,0.831888,0.523815,0.890837,PG
83,De'Aaron Fox,6.028064,5.810007,0.218057,5,0.607942,0.569147,0.529701,0.327572,0.938982,0.890982,0.651389,0.871326,PG
257,Kyle Lowry,5.937895,6.142709,-0.204814,5,0.916170,0.326628,0.708057,0.831471,0.970866,0.837571,0.531439,0.799778,PG
171,Jae'Sean Tate,5.909369,5.899867,0.038006,2,0.545709,0.764539,0.856214,0.250162,0.780520,0.909902,0.785393,0.757390,"PF,SF"
153,Herbert Jones,5.859375,5.859375,0.001449,1,0.507212,0.664663,0.693510,0.746394,0.699519,0.995192,0.912260,0.673077,PF
207,Jordan Clarkson,5.842953,5.667340,0.175612,5,0.902518,0.445016,0.578957,0.772534,0.749487,0.729532,0.320857,0.888608,SG


In [89]:
drafted('Tyrese Maxey')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
42,CJ McCollum,6.128784,6.258050,-0.129266,5,0.950984,0.542452,0.632941,0.564512,0.851964,0.742347,0.660084,0.933124,SG
444,Jamal Murray,6.080538,6.227522,-0.195979,4,0.876877,0.551648,0.602974,0.908174,0.880383,0.831888,0.523815,0.890837,PG
83,De'Aaron Fox,6.028064,5.810007,0.218057,5,0.607942,0.569147,0.529701,0.327572,0.938982,0.890982,0.651389,0.871326,PG
257,Kyle Lowry,5.937895,6.142709,-0.204814,5,0.916170,0.326628,0.708057,0.831471,0.970866,0.837571,0.531439,0.799778,PG
171,Jae'Sean Tate,5.909369,5.899867,0.038006,2,0.545709,0.764539,0.856214,0.250162,0.780520,0.909902,0.785393,0.757390,"PF,SF"
153,Herbert Jones,5.859375,5.859375,0.001449,1,0.507212,0.664663,0.693510,0.746394,0.699519,0.995192,0.912260,0.673077,PF
207,Jordan Clarkson,5.842953,5.667340,0.175612,5,0.902518,0.445016,0.578957,0.772534,0.749487,0.729532,0.320857,0.888608,SG


In [90]:
drafted('Jamal Murray')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
40,Buddy Hield,6.480548,6.422715,0.057833,5,0.986727,0.366107,0.772961,0.869999,0.783113,0.832451,0.606832,0.911289,SG
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
42,CJ McCollum,6.128784,6.258050,-0.129266,5,0.950984,0.542452,0.632941,0.564512,0.851964,0.742347,0.660084,0.933124,SG
83,De'Aaron Fox,6.028064,5.810007,0.218057,5,0.607942,0.569147,0.529701,0.327572,0.938982,0.890982,0.651389,0.871326,PG
257,Kyle Lowry,5.937895,6.142709,-0.204814,5,0.916170,0.326628,0.708057,0.831471,0.970866,0.837571,0.531439,0.799778,PG
171,Jae'Sean Tate,5.909369,5.899867,0.038006,2,0.545709,0.764539,0.856214,0.250162,0.780520,0.909902,0.785393,0.757390,"PF,SF"
153,Herbert Jones,5.859375,5.859375,0.001449,1,0.507212,0.664663,0.693510,0.746394,0.699519,0.995192,0.912260,0.673077,PF
207,Jordan Clarkson,5.842953,5.667340,0.175612,5,0.902518,0.445016,0.578957,0.772534,0.749487,0.729532,0.320857,0.888608,SG
183,Jaren Jackson Jr.,5.842875,5.634737,0.416276,3,0.728573,0.592381,0.713219,0.524994,0.375896,0.684834,0.968068,0.802729,"PF,C"


In [91]:
drafted('Buddy Hield')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
42,CJ McCollum,6.128784,6.258050,-0.129266,5,0.950984,0.542452,0.632941,0.564512,0.851964,0.742347,0.660084,0.933124,SG
83,De'Aaron Fox,6.028064,5.810007,0.218057,5,0.607942,0.569147,0.529701,0.327572,0.938982,0.890982,0.651389,0.871326,PG
257,Kyle Lowry,5.937895,6.142709,-0.204814,5,0.916170,0.326628,0.708057,0.831471,0.970866,0.837571,0.531439,0.799778,PG
171,Jae'Sean Tate,5.909369,5.899867,0.038006,2,0.545709,0.764539,0.856214,0.250162,0.780520,0.909902,0.785393,0.757390,"PF,SF"
153,Herbert Jones,5.859375,5.859375,0.001449,1,0.507212,0.664663,0.693510,0.746394,0.699519,0.995192,0.912260,0.673077,PF
207,Jordan Clarkson,5.842953,5.667340,0.175612,5,0.902518,0.445016,0.578957,0.772534,0.749487,0.729532,0.320857,0.888608,SG
183,Jaren Jackson Jr.,5.842875,5.634737,0.416276,3,0.728573,0.592381,0.713219,0.524994,0.375896,0.684834,0.968068,0.802729,"PF,C"
423,Brook Lopez,5.817483,5.732416,0.113423,4,0.804249,0.585954,0.755010,0.652922,0.466464,0.537802,0.983525,0.782964,C


In [92]:
drafted('OG Anunoby')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
42,CJ McCollum,6.128784,6.258050,-0.129266,5,0.950984,0.542452,0.632941,0.564512,0.851964,0.742347,0.660084,0.933124,SG
83,De'Aaron Fox,6.028064,5.810007,0.218057,5,0.607942,0.569147,0.529701,0.327572,0.938982,0.890982,0.651389,0.871326,PG
257,Kyle Lowry,5.937895,6.142709,-0.204814,5,0.916170,0.326628,0.708057,0.831471,0.970866,0.837571,0.531439,0.799778,PG
171,Jae'Sean Tate,5.909369,5.899867,0.038006,2,0.545709,0.764539,0.856214,0.250162,0.780520,0.909902,0.785393,0.757390,"PF,SF"
153,Herbert Jones,5.859375,5.859375,0.001449,1,0.507212,0.664663,0.693510,0.746394,0.699519,0.995192,0.912260,0.673077,PF
207,Jordan Clarkson,5.842953,5.667340,0.175612,5,0.902518,0.445016,0.578957,0.772534,0.749487,0.729532,0.320857,0.888608,SG
183,Jaren Jackson Jr.,5.842875,5.634737,0.416276,3,0.728573,0.592381,0.713219,0.524994,0.375896,0.684834,0.968068,0.802729,"PF,C"
423,Brook Lopez,5.817483,5.732416,0.113423,4,0.804249,0.585954,0.755010,0.652922,0.466464,0.537802,0.983525,0.782964,C


In [93]:
drafted('RJ Barrett')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
42,CJ McCollum,6.128784,6.258050,-0.129266,5,0.950984,0.542452,0.632941,0.564512,0.851964,0.742347,0.660084,0.933124,SG
83,De'Aaron Fox,6.028064,5.810007,0.218057,5,0.607942,0.569147,0.529701,0.327572,0.938982,0.890982,0.651389,0.871326,PG
257,Kyle Lowry,5.937895,6.142709,-0.204814,5,0.916170,0.326628,0.708057,0.831471,0.970866,0.837571,0.531439,0.799778,PG
171,Jae'Sean Tate,5.909369,5.899867,0.038006,2,0.545709,0.764539,0.856214,0.250162,0.780520,0.909902,0.785393,0.757390,"PF,SF"
153,Herbert Jones,5.859375,5.859375,0.001449,1,0.507212,0.664663,0.693510,0.746394,0.699519,0.995192,0.912260,0.673077,PF
207,Jordan Clarkson,5.842953,5.667340,0.175612,5,0.902518,0.445016,0.578957,0.772534,0.749487,0.729532,0.320857,0.888608,SG
183,Jaren Jackson Jr.,5.842875,5.634737,0.416276,3,0.728573,0.592381,0.713219,0.524994,0.375896,0.684834,0.968068,0.802729,"PF,C"
423,Brook Lopez,5.817483,5.732416,0.113423,4,0.804249,0.585954,0.755010,0.652922,0.466464,0.537802,0.983525,0.782964,C


In [94]:
drafted('CJ McCollum')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
83,De'Aaron Fox,6.028064,5.810007,0.218057,5,0.607942,0.569147,0.529701,0.327572,0.938982,0.890982,0.651389,0.871326,PG
257,Kyle Lowry,5.937895,6.142709,-0.204814,5,0.916170,0.326628,0.708057,0.831471,0.970866,0.837571,0.531439,0.799778,PG
171,Jae'Sean Tate,5.909369,5.899867,0.038006,2,0.545709,0.764539,0.856214,0.250162,0.780520,0.909902,0.785393,0.757390,"PF,SF"
153,Herbert Jones,5.859375,5.859375,0.001449,1,0.507212,0.664663,0.693510,0.746394,0.699519,0.995192,0.912260,0.673077,PF
207,Jordan Clarkson,5.842953,5.667340,0.175612,5,0.902518,0.445016,0.578957,0.772534,0.749487,0.729532,0.320857,0.888608,SG
183,Jaren Jackson Jr.,5.842875,5.634737,0.416276,3,0.728573,0.592381,0.713219,0.524994,0.375896,0.684834,0.968068,0.802729,"PF,C"
423,Brook Lopez,5.817483,5.732416,0.113423,4,0.804249,0.585954,0.755010,0.652922,0.466464,0.537802,0.983525,0.782964,C
232,Keldon Johnson,5.768718,5.668446,0.401089,2,0.719988,0.645379,0.889189,0.397406,0.656292,0.657863,0.500058,0.861832,SF


In [95]:
drafted('Kyle Kuzma')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
83,De'Aaron Fox,6.028064,5.810007,0.218057,5,0.607942,0.569147,0.529701,0.327572,0.938982,0.890982,0.651389,0.871326,PG
257,Kyle Lowry,5.937895,6.142709,-0.204814,5,0.916170,0.326628,0.708057,0.831471,0.970866,0.837571,0.531439,0.799778,PG
171,Jae'Sean Tate,5.909369,5.899867,0.038006,2,0.545709,0.764539,0.856214,0.250162,0.780520,0.909902,0.785393,0.757390,"PF,SF"
153,Herbert Jones,5.859375,5.859375,0.001449,1,0.507212,0.664663,0.693510,0.746394,0.699519,0.995192,0.912260,0.673077,PF
207,Jordan Clarkson,5.842953,5.667340,0.175612,5,0.902518,0.445016,0.578957,0.772534,0.749487,0.729532,0.320857,0.888608,SG
183,Jaren Jackson Jr.,5.842875,5.634737,0.416276,3,0.728573,0.592381,0.713219,0.524994,0.375896,0.684834,0.968068,0.802729,"PF,C"
423,Brook Lopez,5.817483,5.732416,0.113423,4,0.804249,0.585954,0.755010,0.652922,0.466464,0.537802,0.983525,0.782964,C
232,Keldon Johnson,5.768718,5.668446,0.401089,2,0.719988,0.645379,0.889189,0.397406,0.656292,0.657863,0.500058,0.861832,SF


In [96]:
drafted('Julius Randle')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
83,De'Aaron Fox,6.028064,5.810007,0.218057,5,0.607942,0.569147,0.529701,0.327572,0.938982,0.890982,0.651389,0.871326,PG
257,Kyle Lowry,5.937895,6.142709,-0.204814,5,0.916170,0.326628,0.708057,0.831471,0.970866,0.837571,0.531439,0.799778,PG
171,Jae'Sean Tate,5.909369,5.899867,0.038006,2,0.545709,0.764539,0.856214,0.250162,0.780520,0.909902,0.785393,0.757390,"PF,SF"
153,Herbert Jones,5.859375,5.859375,0.001449,1,0.507212,0.664663,0.693510,0.746394,0.699519,0.995192,0.912260,0.673077,PF
207,Jordan Clarkson,5.842953,5.667340,0.175612,5,0.902518,0.445016,0.578957,0.772534,0.749487,0.729532,0.320857,0.888608,SG
183,Jaren Jackson Jr.,5.842875,5.634737,0.416276,3,0.728573,0.592381,0.713219,0.524994,0.375896,0.684834,0.968068,0.802729,"PF,C"
423,Brook Lopez,5.817483,5.732416,0.113423,4,0.804249,0.585954,0.755010,0.652922,0.466464,0.537802,0.983525,0.782964,C
232,Keldon Johnson,5.768718,5.668446,0.401089,2,0.719988,0.645379,0.889189,0.397406,0.656292,0.657863,0.500058,0.861832,SF
151,Harrison Barnes,5.761300,5.699042,0.062258,5,0.821531,0.553416,0.835666,0.697421,0.690806,0.622261,0.341715,0.880822,"PF,PF-SF"


In [97]:
drafted('Marcus Smart')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
83,De'Aaron Fox,6.028064,5.810007,0.218057,5,0.607942,0.569147,0.529701,0.327572,0.938982,0.890982,0.651389,0.871326,PG
257,Kyle Lowry,5.937895,6.142709,-0.204814,5,0.916170,0.326628,0.708057,0.831471,0.970866,0.837571,0.531439,0.799778,PG
171,Jae'Sean Tate,5.909369,5.899867,0.038006,2,0.545709,0.764539,0.856214,0.250162,0.780520,0.909902,0.785393,0.757390,"PF,SF"
153,Herbert Jones,5.859375,5.859375,0.001449,1,0.507212,0.664663,0.693510,0.746394,0.699519,0.995192,0.912260,0.673077,PF
207,Jordan Clarkson,5.842953,5.667340,0.175612,5,0.902518,0.445016,0.578957,0.772534,0.749487,0.729532,0.320857,0.888608,SG
183,Jaren Jackson Jr.,5.842875,5.634737,0.416276,3,0.728573,0.592381,0.713219,0.524994,0.375896,0.684834,0.968068,0.802729,"PF,C"
423,Brook Lopez,5.817483,5.732416,0.113423,4,0.804249,0.585954,0.755010,0.652922,0.466464,0.537802,0.983525,0.782964,C
232,Keldon Johnson,5.768718,5.668446,0.401089,2,0.719988,0.645379,0.889189,0.397406,0.656292,0.657863,0.500058,0.861832,SF
151,Harrison Barnes,5.761300,5.699042,0.062258,5,0.821531,0.553416,0.835666,0.697421,0.690806,0.622261,0.341715,0.880822,"PF,PF-SF"


In [98]:
drafted('Devin Vassell')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
83,De'Aaron Fox,6.028064,5.810007,0.218057,5,0.607942,0.569147,0.529701,0.327572,0.938982,0.890982,0.651389,0.871326,PG
257,Kyle Lowry,5.937895,6.142709,-0.204814,5,0.916170,0.326628,0.708057,0.831471,0.970866,0.837571,0.531439,0.799778,PG
171,Jae'Sean Tate,5.909369,5.899867,0.038006,2,0.545709,0.764539,0.856214,0.250162,0.780520,0.909902,0.785393,0.757390,"PF,SF"
153,Herbert Jones,5.859375,5.859375,0.001449,1,0.507212,0.664663,0.693510,0.746394,0.699519,0.995192,0.912260,0.673077,PF
207,Jordan Clarkson,5.842953,5.667340,0.175612,5,0.902518,0.445016,0.578957,0.772534,0.749487,0.729532,0.320857,0.888608,SG
183,Jaren Jackson Jr.,5.842875,5.634737,0.416276,3,0.728573,0.592381,0.713219,0.524994,0.375896,0.684834,0.968068,0.802729,"PF,C"
423,Brook Lopez,5.817483,5.732416,0.113423,4,0.804249,0.585954,0.755010,0.652922,0.466464,0.537802,0.983525,0.782964,C
232,Keldon Johnson,5.768718,5.668446,0.401089,2,0.719988,0.645379,0.889189,0.397406,0.656292,0.657863,0.500058,0.861832,SF
151,Harrison Barnes,5.761300,5.699042,0.062258,5,0.821531,0.553416,0.835666,0.697421,0.690806,0.622261,0.341715,0.880822,"PF,PF-SF"


In [99]:
drafted('Jalen Green')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
83,De'Aaron Fox,6.028064,5.810007,0.218057,5,0.607942,0.569147,0.529701,0.327572,0.938982,0.890982,0.651389,0.871326,PG
257,Kyle Lowry,5.937895,6.142709,-0.204814,5,0.916170,0.326628,0.708057,0.831471,0.970866,0.837571,0.531439,0.799778,PG
171,Jae'Sean Tate,5.909369,5.899867,0.038006,2,0.545709,0.764539,0.856214,0.250162,0.780520,0.909902,0.785393,0.757390,"PF,SF"
153,Herbert Jones,5.859375,5.859375,0.001449,1,0.507212,0.664663,0.693510,0.746394,0.699519,0.995192,0.912260,0.673077,PF
207,Jordan Clarkson,5.842953,5.667340,0.175612,5,0.902518,0.445016,0.578957,0.772534,0.749487,0.729532,0.320857,0.888608,SG
183,Jaren Jackson Jr.,5.842875,5.634737,0.416276,3,0.728573,0.592381,0.713219,0.524994,0.375896,0.684834,0.968068,0.802729,"PF,C"
423,Brook Lopez,5.817483,5.732416,0.113423,4,0.804249,0.585954,0.755010,0.652922,0.466464,0.537802,0.983525,0.782964,C
232,Keldon Johnson,5.768718,5.668446,0.401089,2,0.719988,0.645379,0.889189,0.397406,0.656292,0.657863,0.500058,0.861832,SF
151,Harrison Barnes,5.761300,5.699042,0.062258,5,0.821531,0.553416,0.835666,0.697421,0.690806,0.622261,0.341715,0.880822,"PF,PF-SF"


In [100]:
drafted('P.J. Washington')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
83,De'Aaron Fox,6.028064,5.810007,0.218057,5,0.607942,0.569147,0.529701,0.327572,0.938982,0.890982,0.651389,0.871326,PG
257,Kyle Lowry,5.937895,6.142709,-0.204814,5,0.916170,0.326628,0.708057,0.831471,0.970866,0.837571,0.531439,0.799778,PG
171,Jae'Sean Tate,5.909369,5.899867,0.038006,2,0.545709,0.764539,0.856214,0.250162,0.780520,0.909902,0.785393,0.757390,"PF,SF"
153,Herbert Jones,5.859375,5.859375,0.001449,1,0.507212,0.664663,0.693510,0.746394,0.699519,0.995192,0.912260,0.673077,PF
207,Jordan Clarkson,5.842953,5.667340,0.175612,5,0.902518,0.445016,0.578957,0.772534,0.749487,0.729532,0.320857,0.888608,SG
183,Jaren Jackson Jr.,5.842875,5.634737,0.416276,3,0.728573,0.592381,0.713219,0.524994,0.375896,0.684834,0.968068,0.802729,"PF,C"
423,Brook Lopez,5.817483,5.732416,0.113423,4,0.804249,0.585954,0.755010,0.652922,0.466464,0.537802,0.983525,0.782964,C
232,Keldon Johnson,5.768718,5.668446,0.401089,2,0.719988,0.645379,0.889189,0.397406,0.656292,0.657863,0.500058,0.861832,SF
151,Harrison Barnes,5.761300,5.699042,0.062258,5,0.821531,0.553416,0.835666,0.697421,0.690806,0.622261,0.341715,0.880822,"PF,PF-SF"


In [101]:
drafted('Mike Conley')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
83,De'Aaron Fox,6.028064,5.810007,0.218057,5,0.607942,0.569147,0.529701,0.327572,0.938982,0.890982,0.651389,0.871326,PG
257,Kyle Lowry,5.937895,6.142709,-0.204814,5,0.916170,0.326628,0.708057,0.831471,0.970866,0.837571,0.531439,0.799778,PG
171,Jae'Sean Tate,5.909369,5.899867,0.038006,2,0.545709,0.764539,0.856214,0.250162,0.780520,0.909902,0.785393,0.757390,"PF,SF"
153,Herbert Jones,5.859375,5.859375,0.001449,1,0.507212,0.664663,0.693510,0.746394,0.699519,0.995192,0.912260,0.673077,PF
207,Jordan Clarkson,5.842953,5.667340,0.175612,5,0.902518,0.445016,0.578957,0.772534,0.749487,0.729532,0.320857,0.888608,SG
183,Jaren Jackson Jr.,5.842875,5.634737,0.416276,3,0.728573,0.592381,0.713219,0.524994,0.375896,0.684834,0.968068,0.802729,"PF,C"
423,Brook Lopez,5.817483,5.732416,0.113423,4,0.804249,0.585954,0.755010,0.652922,0.466464,0.537802,0.983525,0.782964,C
232,Keldon Johnson,5.768718,5.668446,0.401089,2,0.719988,0.645379,0.889189,0.397406,0.656292,0.657863,0.500058,0.861832,SF
151,Harrison Barnes,5.761300,5.699042,0.062258,5,0.821531,0.553416,0.835666,0.697421,0.690806,0.622261,0.341715,0.880822,"PF,PF-SF"


In [102]:
drafted('Keldon Johnson')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
14,Andrew Wiggins,6.264315,6.169491,0.094824,5,0.830623,0.488220,0.744570,0.216075,0.726443,0.819537,0.872884,0.911644,"SF,PF"
83,De'Aaron Fox,6.028064,5.810007,0.218057,5,0.607942,0.569147,0.529701,0.327572,0.938982,0.890982,0.651389,0.871326,PG
257,Kyle Lowry,5.937895,6.142709,-0.204814,5,0.916170,0.326628,0.708057,0.831471,0.970866,0.837571,0.531439,0.799778,PG
171,Jae'Sean Tate,5.909369,5.899867,0.038006,2,0.545709,0.764539,0.856214,0.250162,0.780520,0.909902,0.785393,0.757390,"PF,SF"
153,Herbert Jones,5.859375,5.859375,0.001449,1,0.507212,0.664663,0.693510,0.746394,0.699519,0.995192,0.912260,0.673077,PF
207,Jordan Clarkson,5.842953,5.667340,0.175612,5,0.902518,0.445016,0.578957,0.772534,0.749487,0.729532,0.320857,0.888608,SG
183,Jaren Jackson Jr.,5.842875,5.634737,0.416276,3,0.728573,0.592381,0.713219,0.524994,0.375896,0.684834,0.968068,0.802729,"PF,C"
423,Brook Lopez,5.817483,5.732416,0.113423,4,0.804249,0.585954,0.755010,0.652922,0.466464,0.537802,0.983525,0.782964,C
151,Harrison Barnes,5.761300,5.699042,0.062258,5,0.821531,0.553416,0.835666,0.697421,0.690806,0.622261,0.341715,0.880822,"PF,PF-SF"
341,Robert Covington,5.747632,5.738948,0.008685,5,0.859279,0.278903,0.786730,0.656050,0.484026,0.957694,0.929204,0.627269,"PF,PF-C,SF"


In [103]:
drafted('Andrew Wiggins')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
83,De'Aaron Fox,6.028064,5.810007,0.218057,5,0.607942,0.569147,0.529701,0.327572,0.938982,0.890982,0.651389,0.871326,PG
257,Kyle Lowry,5.937895,6.142709,-0.204814,5,0.916170,0.326628,0.708057,0.831471,0.970866,0.837571,0.531439,0.799778,PG
171,Jae'Sean Tate,5.909369,5.899867,0.038006,2,0.545709,0.764539,0.856214,0.250162,0.780520,0.909902,0.785393,0.757390,"PF,SF"
153,Herbert Jones,5.859375,5.859375,0.001449,1,0.507212,0.664663,0.693510,0.746394,0.699519,0.995192,0.912260,0.673077,PF
207,Jordan Clarkson,5.842953,5.667340,0.175612,5,0.902518,0.445016,0.578957,0.772534,0.749487,0.729532,0.320857,0.888608,SG
183,Jaren Jackson Jr.,5.842875,5.634737,0.416276,3,0.728573,0.592381,0.713219,0.524994,0.375896,0.684834,0.968068,0.802729,"PF,C"
423,Brook Lopez,5.817483,5.732416,0.113423,4,0.804249,0.585954,0.755010,0.652922,0.466464,0.537802,0.983525,0.782964,C
151,Harrison Barnes,5.761300,5.699042,0.062258,5,0.821531,0.553416,0.835666,0.697421,0.690806,0.622261,0.341715,0.880822,"PF,PF-SF"
341,Robert Covington,5.747632,5.738948,0.008685,5,0.859279,0.278903,0.786730,0.656050,0.484026,0.957694,0.929204,0.627269,"PF,PF-C,SF"
426,Collin Sexton,5.703447,5.565554,0.275786,3,0.761268,0.579459,0.491460,0.754968,0.821026,0.745178,0.203045,0.938182,"SG,PG"


In [104]:
drafted('Collin Sexton')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
83,De'Aaron Fox,6.028064,5.810007,0.218057,5,0.607942,0.569147,0.529701,0.327572,0.938982,0.890982,0.651389,0.871326,PG
257,Kyle Lowry,5.937895,6.142709,-0.204814,5,0.916170,0.326628,0.708057,0.831471,0.970866,0.837571,0.531439,0.799778,PG
171,Jae'Sean Tate,5.909369,5.899867,0.038006,2,0.545709,0.764539,0.856214,0.250162,0.780520,0.909902,0.785393,0.757390,"PF,SF"
153,Herbert Jones,5.859375,5.859375,0.001449,1,0.507212,0.664663,0.693510,0.746394,0.699519,0.995192,0.912260,0.673077,PF
207,Jordan Clarkson,5.842953,5.667340,0.175612,5,0.902518,0.445016,0.578957,0.772534,0.749487,0.729532,0.320857,0.888608,SG
183,Jaren Jackson Jr.,5.842875,5.634737,0.416276,3,0.728573,0.592381,0.713219,0.524994,0.375896,0.684834,0.968068,0.802729,"PF,C"
423,Brook Lopez,5.817483,5.732416,0.113423,4,0.804249,0.585954,0.755010,0.652922,0.466464,0.537802,0.983525,0.782964,C
151,Harrison Barnes,5.761300,5.699042,0.062258,5,0.821531,0.553416,0.835666,0.697421,0.690806,0.622261,0.341715,0.880822,"PF,PF-SF"
341,Robert Covington,5.747632,5.738948,0.008685,5,0.859279,0.278903,0.786730,0.656050,0.484026,0.957694,0.929204,0.627269,"PF,PF-C,SF"
235,Kelly Oubre Jr.,5.696938,5.704367,-0.007429,5,0.805084,0.415998,0.755211,0.437646,0.426752,0.865296,0.787381,0.818911,SF
